# Импорт библиотек

In [2]:
import os
import re

import pdfplumber
import joblib
import pandas as pd
from tqdm import tqdm
from datetime import timedelta
import numpy as np
import random

In [3]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.decomposition import PCA

c:\Users\filat\Documents\projects\my_champion_app\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
tqdm.pandas()
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
RANDOM_STATE = 1206

# Загрузка данных

In [48]:
total_scores = pd.read_csv("../data/raw/total_scores.csv")
tournament_scores = pd.read_csv("../data/raw/tournament_scores.csv")
tournaments = pd.read_csv("../data/raw/tournaments.csv")
units = pd.read_csv("../data/raw/units.csv")

In [49]:
total_scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21301 entries, 0 to 21300
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   21301 non-null  int64  
 1   unit_id              21301 non-null  int64  
 2   tournament_id        21301 non-null  int64  
 3   base_score           21301 non-null  float64
 4   components_score     21301 non-null  float64
 5   total_score          21301 non-null  float64
 6   elements_score       21301 non-null  float64
 7   decreasings_score    21301 non-null  float64
 8   starting_place       21301 non-null  int64  
 9   place                21301 non-null  int64  
 10  segment_name         21284 non-null  object 
 11  info                 20720 non-null  object 
 12  overall_place        21301 non-null  int64  
 13  overall_total_score  21284 non-null  float64
 14  overall_place_str    10814 non-null  object 
dtypes: float64(6), int64(6), object(3)
m

In [50]:
total_scores.head(3)

,id,unit_id,tournament_id,base_score,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str
0,442027,304,4785,47.2,43.47,102.70,59.23,0.0,17,1,Короткая программа,x Надбавка за прыжки во второй половине програ...,2,293.74,2
1,442028,604,4785,45.4,46.71,101.19,54.48,0.0,18,2,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,1,294.75,1
2,442029,409,4785,44.0,46.82,99.20,52.38,0.0,6,3,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,4,285.57,4


In [51]:
tournament_scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172158 entries, 0 to 172157
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              172158 non-null  int64  
 1   total_score_id  172158 non-null  int64  
 2   title           172158 non-null  object 
 3   decrease        41185 non-null   object 
 4   base_score      172158 non-null  float64
 5   goe             172158 non-null  float64
 6   avg_score       172158 non-null  float64
dtypes: float64(3), int64(2), object(2)
memory usage: 9.2+ MB


In [52]:
tournament_scores.head(3)

,id,total_score_id,title,decrease,base_score,goe,avg_score
0,1,1,2A,NaN,3.3,0.66,3.96
1,2,1,3F+3Lo,NaN,10.2,-0.11,10.09
2,3,1,3Lz,NaN,0.0,-1.30,5.19


In [53]:
tournaments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142 entries, 0 to 141
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          142 non-null    int64  
 1   date_start  142 non-null    object 
 2   date_end    142 non-null    object 
 3   origin_id   142 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 4.6+ KB


In [54]:
tournaments.head(3)

,id,date_start,date_end,origin_id
0,1,2090-11-29,2090-12-01,2.0
1,2,2091-03-06,2091-03-10,1.0
2,3,2090-10-05,2090-10-08,2.0


In [55]:
units.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4596 entries, 0 to 4595
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         4596 non-null   int64  
 1   color      4595 non-null   object 
 2   school_id  4007 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 107.8+ KB


In [56]:
units.head(3)

,id,color,school_id
0,9474,green,244.0
1,733,green,203.0
2,734,green,235.0


# Объединение таблиц

In [57]:
total_scores = total_scores.set_index("id")

In [58]:
units = units.set_index("id")

In [59]:
tournaments = tournaments.set_index("id")

In [60]:
data = tournament_scores.join(
    total_scores, on="total_score_id", rsuffix="_total_scores"
)
data = data.join(units, on="unit_id", rsuffix="_units")
data = data.join(tournaments, on="tournament_id", rsuffix="_tournaments")

In [61]:
data.head(1)

,id,total_score_id,title,decrease,base_score,goe,avg_score,unit_id,tournament_id,base_score_total_scores,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,color,school_id,date_start,date_end,origin_id
0,1,1,2A,NaN,3.3,0.66,3.96,1,1,31.99,24.67,56.86,33.19,0.0,2,5,Короткая программа,x Надбавка за прыжки во второй половине програ...,4,164.44,NaN,green,198.0,2090-11-29,2090-12-01,2.0


# Справочная информация

Распарсим значения 'base_score' для каждого элемента из справочной информации

In [62]:
file_path = "../data/mats/base_scores_dict.joblib"

if os.path.isfile(file_path):
    base_scores_dict = joblib.load(file_path)
    get_base_scores_dict = False
else:
    get_base_scores_dict = True

In [65]:
if get_base_scores_dict:
    # if True:
    pdf_path = "../archive/2475_SP_SOV_2022-23.pdf"

    with pdfplumber.open(pdf_path) as pdf:
        all_tables = []
        # Обработка каждой страницы
        for page in pdf.pages:
            # Извлечение таблиц со страницы
            tables = page.extract_tables()
            for table in tables:
                # Преобразование таблицы в DataFrame и добавление в список
                df = pd.DataFrame(table)
                all_tables.append(df)

    base_scores_dict = pd.concat(all_tables[1:17], ignore_index=True)
    base_scores_dict = base_scores_dict[base_scores_dict[0] != ""]
    base_scores_dict = base_scores_dict.reset_index(drop=True)
    base_scores_dict = base_scores_dict[[0, 6]]
    base_scores_dict = base_scores_dict.rename(columns={0: "item", 6: "base_score"})
    base_scores_dict["base_score"] = (
        base_scores_dict["base_score"].str.replace(",", ".").astype(float)
    )
    base_scores_dict.loc[134, "item"] = "3A"
    base_scores_dict.loc[142, "item"] = "3Aq"
    # Добавлен элемент российской системы, введенный с февраля 2023 года
    base_scores_dict.loc[len(base_scores_dict)] = ["ChSpl1", 1.5]
    # Добавлен элемент, возможно новичков, не применяемый на профессиональных соревнованиях
    base_scores_dict.loc[len(base_scores_dict)] = ["1W", 0.2]

    joblib.dump(base_scores_dict, file_path)
    get_base_scores_dict = False

Сформируем представление элементов на естественном языке с учетом ошибок

In [66]:
file_path = "../data/mats/nlp_dict.joblib"

if os.path.isfile(file_path):
    nlp_dict = joblib.load(file_path)
    get_nlp_dict = False
else:
    get_nlp_dict = True

In [68]:
if get_nlp_dict:
    # if True:
    pdf_path = "../archive/Elemente-Liste2023_24.pdf"

    with pdfplumber.open(pdf_path) as pdf:
        all_tables = []
        # Обработка каждой страницы
        for page in pdf.pages:
            # Извлечение таблиц со страницы
            tables = page.extract_tables()
            for table in tables:
                # Преобразование таблицы в DataFrame и добавление в список
                df = pd.DataFrame(table)
                all_tables.append(df)

    nlp_dict = pd.concat(all_tables[0:3], ignore_index=True)
    nlp_dict = pd.concat([nlp_dict, all_tables[19]], ignore_index=True)
    nlp_dict = nlp_dict.rename(columns={0: "item", 1: "nlp_item"})
    nlp_dict["item"] = nlp_dict["item"].str.replace("+ ", "")

    steps_spins = [
        "USp",
        "LSp",
        "CSp",
        "SSp",
        "FUSp",
        "FLSp",
        "FCSp",
        "FSSp",
        "CUSp",
        "CLSp",
        "CCSp",
        "CSSp",
        "FCUSp",
        "FCLSp",
        "FCCSp",
        "FCSSp",
        "CoSp",
        "CCoSp",
        "FCoSp",
        "FCCoSp",
        "StSq",
    ]

    for item in steps_spins:
        for i in range(0, 5):
            levels = ["B", "1", "2", "3", "4"]
            nlp = nlp_dict[nlp_dict["item"] == item].iloc[0, 1]
            add_nlp = [
                item + levels[i],
                " " + str(nlp) + " (" + "level" + " " + levels[i] + ")",
            ]
            nlp_dict.loc[len(nlp_dict)] = add_nlp

    nlp_dict.loc[len(nlp_dict)] = ["ChSq1", "Choreo Sequence"]
    nlp_dict.loc[len(nlp_dict)] = ["ChSpl1", "Choreo sliding movement"]
    nlp_dict.loc[len(nlp_dict)] = ["1W", "Waltz"]
    nlp_dict.loc[len(nlp_dict)] = ["0", "There is no element"]

In [69]:
nlp_dict[nlp_dict["item"] == "2A"]["nlp_item"]

12    Double Axel
Name: nlp_item, dtype: object

In [70]:
nlp_dict.head()

,item,nlp_item
0,1T,Single Toeloop
1,1S,Single Salchow
2,1Lo,Single Loop
3,1Eu,Single Euler (only used in jump combinations)
4,1F,Single Flip


In [71]:
nlp_dict.shape

(168, 2)

# Изучение и предобработка данных

## Загрузка данных, если существуют для сокращения времени работы

In [72]:
file_path = "../data/processed/processed_data.joblib"

if os.path.isfile(file_path):
    data = joblib.load(file_path)
    get_processed_data = False
else:
    get_processed_data = True

## Предварительные правки дата-сета

Первоначально внесем небольшие изменения, искажающие данные (проблемы были обнаружены в ходе экспериментов)

In [73]:
if get_processed_data:
    # ВАЖНО - РЕГИСТР
    data.loc[124247, "title"] = "FSSp2"
    # ВАЖНО - КОДИРОВКА
    data["title"] = data["title"].str.replace("SЕQ", "SEQ")

## Парсер элементов

Распарсим все элементы, выполненные фигуристами с указанием их "аттрибутов": ошибок и штрафов

In [74]:
def get_sequence(elements):
    sequence_pattern = r"""(?P<item>(\d[AEuTSLoFz]+)
                                   |([USCLFo]+p[\dB]?)
                                   |([StTCh]+q[\dB]?)
                                   |(ChSpl1)
                                   |([SEQRPCOMB]+))
                            (?P<attr>[<!Vbqe]*)
                            (?P<fail>[\*]{1})*"""
    get_items = re.compile(sequence_pattern, re.X)
    get_attrs = re.compile(r"<<|[VBqe<!]|\d")
    sequence = elements.split("+")

    def parse_element(x):
        try:
            res = get_items.search(x).groupdict()
        except AttributeError:
            res = {"item": "FAIL", "attr": None, "fail": None}
        finally:
            if res["attr"] is not None:
                attrs = get_attrs.findall(res["attr"])
                res["attr"] = attrs
        return res

    result = [parse_element(x) for x in sequence]

    return result

In [75]:
if get_processed_data:
    data["sequences"] = data["title"].apply(get_sequence)

data[["title", "sequences", "base_score", "info"]].head(5)

,title,sequences,base_score,info
0,2A,"[{'item': '2A', 'attr': [], 'fail': None}]",3.3,x Надбавка за прыжки во второй половине програ...
1,3F+3Lo,"[{'item': '3F', 'attr': [], 'fail': None}, {'i...",10.2,x Надбавка за прыжки во второй половине програ...
2,3Lz,"[{'item': '3Lz', 'attr': [], 'fail': None}]",0.0,x Надбавка за прыжки во второй половине програ...
3,CCoSp4,"[{'item': 'CCoSp4', 'attr': [], 'fail': None}]",3.5,x Надбавка за прыжки во второй половине програ...
4,FCSp4,"[{'item': 'FCSp4', 'attr': [], 'fail': None}]",3.2,x Надбавка за прыжки во второй половине програ...


## "Чистые" каскады

Сформируем столбец с чистыми каскадами (без указания ошибок)

In [76]:
def get_cascade(sequences):
    return "+".join([x["item"] for x in sequences])

In [77]:
if get_processed_data:
    data["cascade"] = data["sequences"].progress_apply(get_cascade)

100%|██████████| 172158/172158 [00:00<00:00, 882868.32it/s]


Проверим, имеются ли элементы, которые не удалось распарсить - ошибки в написании и т.д.

In [78]:
errors = data[data["cascade"].str.contains("FAIL")]
display(len(errors))
if len(errors) > 0:
    errors.sample(5, random_state=RANDOM_STATE)

183

В записях элементов представлены ошибки. Не будем использовать такие строки.

In [79]:
data = data[~data["cascade"].str.contains("FAIL")]

## NLP преобразование

Сформируем столбцы с преобразованиями наборов элементов на естественный язык.

In [80]:
def get_title_NLP(sequences, nlp_dict, clear_cascade=False):
    if sequences == 0:
        return 0
    else:
        sequence = []
        for x in sequences:
            try:
                element_nlp = [nlp_dict[nlp_dict["item"] == x["item"]].iloc[0, 1]]
                errors_nlp = []
                if not clear_cascade:
                    errors_nlp = []
                    for error in x["attr"]:
                        errors_nlp.append(
                            nlp_dict[nlp_dict["item"] == error].iloc[0, 1]
                        )
                    if x["fail"] is not None:
                        errors_nlp.append("fail")
                sequence.append(" ".join(element_nlp + errors_nlp))
            except IndexError:
                return "Unknown element"
        return " and ".join(sequence)

In [81]:
if get_processed_data:
    # if True:
    data["title_nlp"] = data["sequences"].progress_apply(
        get_title_NLP, args=(nlp_dict,)
    )

100%|██████████| 171975/171975 [00:56<00:00, 3039.26it/s]


In [82]:
data[["title", "title_nlp"]].head(15)

,title,title_nlp
0,2A,Double Axel
1,3F+3Lo,Triple Flip and Triple Loop
2,3Lz,Triple Lutz
3,CCoSp4,Change Foot Combination Spin (level 4)
4,FCSp4,Flying Camel Spin (level 4)
5,LSp4,Layback Spin (level 4)
6,StSq2,Step Sequence (level 2)
7,2A,Double Axel
8,3F,Triple Flip
9,3Fq+3Loq,Triple Flip jump landed on the quarter and Tri...


In [83]:
data[data["title_nlp"] == "Unknown element"].head(5)

,id,total_score_id,title,decrease,base_score,goe,avg_score,unit_id,tournament_id,base_score_total_scores,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,color,school_id,date_start,date_end,origin_id,sequences,cascade,title_nlp
282,283,274,S,F,0.0,0.0,0.0,16,1,39.50,40.29,82.99,43.70,0.0,24,2,Произвольная программа,x Надбавка за прыжки во второй половине програ...,2,135.73,NaN,green,212.0,2090-11-29,2090-12-01,2.0,"[{'item': 'S', 'attr': [], 'fail': None}]",S,Unknown element
759,760,750,S,F,0.0,0.0,0.0,44,1,29.10,39.16,69.40,31.24,0.0,12,16,Произвольная программа,x Надбавка за прыжки во второй половине програ...,17,114.77,NaN,green,139.0,2090-11-29,2090-12-01,2.0,"[{'item': 'S', 'attr': [], 'fail': None}]",S,Unknown element
1061,2427,2424,ChSpl,F,0.0,0.0,0.0,163,2,3.80,10.01,13.53,4.02,0.0,4,26,Произвольная программа,"Мастерство катания@@@1.50@@@3,25@@@3,50@@@3,50...",26,13.53,NaN,green,111.0,2091-03-06,2091-03-10,1.0,"[{'item': 'C', 'attr': [], 'fail': None}]",C,Unknown element
1559,162562,442829,Sp*,F,0.0,0.0,0.0,1331,4793,29.25,31.90,66.26,37.36,-3.0,20,2,Короткая программа,Нарушения в костюме:@@@(1 of 6),2,200.75,2,green,82.0,2091-12-09,2091-12-11,2.0,"[{'item': 'Sp', 'attr': [], 'fail': '*'}]",Sp,Unknown element
2332,2343,2340,ChSpl,NaN,0.0,0.0,0.0,149,2,5.50,11.63,17.61,5.98,0.0,19,11,Произвольная программа,"Мастерство катания@@@1.50@@@4,25@@@3,50@@@3,75...",11,17.61,NaN,green,27.0,2091-03-06,2091-03-10,1.0,"[{'item': 'C', 'attr': [], 'fail': None}]",C,Unknown element


In [84]:
data[data["title_nlp"] == "Unknown element"]["title"].unique()

array(['S', 'ChSpl', 'Sp*', 'Sp', 'S*', 'S*+COMBO'], dtype=object)

Вероятно, это неправильно записанные steps, spins и slide. Однако, количество очков за их выполнение - 0.0. Предполагается, что указанные записи не принесут пользы при обучении. При этом, часть информации о выступлении спортсмена на соревновании будет потеряно, в частности, последовательность выполнения элементов. В этой связи последовательность выполнения элементов мы не будем оценивать. А все ошибочные записи удалим. 

Возможно, это повлияет на данные, как на временной ряд. Однако, с данными такого качества обучение ожидается еще хуже. Поэтому, оставим данные только хорошего качества.

In [85]:
data = data[~(data["title_nlp"] == "Unknown element")]

In [86]:
data.shape

(171730, 29)

Сформируем столбец с nlp описаниями "чистых" каскадов.

In [87]:
if get_processed_data:
    data["cascade_nlp"] = data["sequences"].progress_apply(
        get_title_NLP,
        args=(
            nlp_dict,
            True,
        ),
    )

100%|██████████| 171730/171730 [00:47<00:00, 3620.94it/s]


## Надбавки

Замечено, что в столбце 'base_score' имеются значения, равные 0.0 для строк, которые имеют записи в поле 'info': "x Надбавка за прыжки во второй половине программы (10%)". Это означает, что необходимо рассчитать base_score с учетом повышающего коэффициента 1.1.

Определим строки, для которых предполагается надбавка. Кроме того, этот столбец также будет демонстрировать повышенную сложность элемента.

In [88]:
def get_multiply(row: pd.Series) -> int:
    original_base_score = row["base_score"]
    info = row["info"]
    if (
        isinstance(info, str)
        and "Надбавка за прыжки во второй половине программы" in info
        and original_base_score == 0.0
    ):
        return 1
    return 0

In [89]:
if get_processed_data:
    data["multiply"] = data.progress_apply(get_multiply, axis=1)
data["multiply"].sum()

100%|██████████| 171730/171730 [00:01<00:00, 109277.94it/s]


30246

30246 строк с надбавкой. Выведем несколько и проверим.

In [91]:
if get_processed_data:
    display(
        data[data["multiply"] == 1][
            ["title", "base_score", "multiply", "goe", "avg_score"]
        ].loc[[109822, 18943, 40127, 17630, 101858]]
    )

,title,base_score,multiply,goe,avg_score
109822,3Lz+1Eu+2S,0.0,1,0.98,9.45
18943,3Lo,0.0,1,0.65,6.04
40127,3F!,0.0,1,-1.06,4.77
17630,3S<,0.0,1,-0.69,3.09
101858,2Lz+2Lo+1A+SEQ,0.0,1,0.21,5.60


Расчеты показывают верность выводов:
- 3Lz+1Eu+2S -> (5.90 + 0.50 + 1.30)*1.1 + 0.98 = **9.45**
- 3Lo -> 4.90*1.1 + 0.65 = **6.04**
- 3F! -> 5.30*1.1 - 1.06 = **4.77**
- 3S< -> 3.44*1.1 - 0.69 = **3.094**
- 2Lz+2Lo+1A+SEQ -> (2.10 + 1.70 + 1.10\*1.0)*1.1 + 0.21 = **5.60**

В последнем примере необходимо отметить, что отметка **SEQ** не принесла никаких изменений. 

**SEQ (Sequence)** указывает, что прыжки выполнены не в плотной комбинации, а с некоторым промежутком между ними. Это означает, что между прыжками могут быть шаги или другие элементы, но прыжки всё равно считаются одной последовательностью.

"+SEQ" влияет на судейскую оценку, поскольку последовательности прыжков оцениваются по-другому по сравнению с комбинациями, где прыжки выполняются подряд без промежутков. Поиск в интернете дал ответ, что такая последовательность прыжков оценивается ниже, чем комбинация из тех же прыжков, из-за применения скидки 20% к каждому прыжку после первого в последовательности.

Однако, в дата-сете такого снижения очков не замечено. В связи с этим, в дальнейших расчетах отметка "+SEQ" не вносила изменений в расчетное количество очков.

*Можно уточнить этот момент у заказчика*.

## Изучение поля "segment_name"

In [92]:
data["segment_name"].value_counts()

segment_name
Произвольная программа     91729
Короткая программа         53467
Произвольная программа.    16989
Элементы                    3519
Пpoизвoльнaя пpoгpaммa      1580
Прыжки                       852
Кoроткая программа           700
Кopoткaя пpoгpaммa           659
Произвольнaя программа       473
Прoизвольная прoграмма       427
Kopoткaя пpoгpaммa           377
Элeмeнты                     341
Пpoизвольная программа       315
Коpоткая программа           126
Интерпретация.               102
Пpoизвольнaя программа        70
Интерпретация                  4
Name: count, dtype: int64

Очень много неявных дубликатов, элементы разных уровней фигурного катания: любительское и профессиональное. Заметно, что данных профессионального уровня значительно больше. Сосредоточимся на них.

In [93]:
if get_processed_data:
    data["segment_name"] = data["segment_name"].str.replace(".", "")
data["segment_name"].value_counts()

segment_name
Произвольная программа    108718
Короткая программа         53467
Элементы                    3519
Пpoизвoльнaя пpoгpaммa      1580
Прыжки                       852
Кoроткая программа           700
Кopoткaя пpoгpaммa           659
Произвольнaя программа       473
Прoизвольная прoграмма       427
Kopoткaя пpoгpaммa           377
Элeмeнты                     341
Пpoизвольная программа       315
Коpоткая программа           126
Интерпретация                106
Пpoизвольнaя программа        70
Name: count, dtype: int64

In [94]:
if get_processed_data:
    data["segment_name"] = data["segment_name"].str.replace(
        "Пpoизвoльнaя пpoгpaммa", "Произвольная программа"
    )
    data["segment_name"] = data["segment_name"].str.replace(
        "Произвольнaя программа", "Произвольная программа"
    )
    data["segment_name"] = data["segment_name"].str.replace(
        "Прoизвольная прoграмма", "Произвольная программа"
    )
    data["segment_name"] = data["segment_name"].str.replace(
        "Пpoизвольная программа", "Произвольная программа"
    )
    data["segment_name"] = data["segment_name"].str.replace(
        "Пpoизвольнaя программа", "Произвольная программа"
    )
data["segment_name"].value_counts()

segment_name
Произвольная программа    111583
Короткая программа         53467
Элементы                    3519
Прыжки                       852
Кoроткая программа           700
Кopoткaя пpoгpaммa           659
Kopoткaя пpoгpaммa           377
Элeмeнты                     341
Коpоткая программа           126
Интерпретация                106
Name: count, dtype: int64

In [95]:
if get_processed_data:
    data["segment_name"] = data["segment_name"].str.replace(
        "Кoроткая программа", "Короткая программа"
    )
    data["segment_name"] = data["segment_name"].str.replace(
        "Кopoткaя пpoгpaммa", "Короткая программа"
    )
    data["segment_name"] = data["segment_name"].str.replace(
        "Kopoткaя пpoгpaммa", "Короткая программа"
    )
    data["segment_name"] = data["segment_name"].str.replace(
        "Коpоткая программа", "Короткая программа"
    )
data["segment_name"].value_counts()

segment_name
Произвольная программа    111583
Короткая программа         55329
Элементы                    3519
Прыжки                       852
Элeмeнты                     341
Интерпретация                106
Name: count, dtype: int64

In [96]:
if get_processed_data:
    data["segment_name"] = data["segment_name"].str.replace("Элeмeнты", "Элементы")
data["segment_name"].value_counts()

segment_name
Произвольная программа    111583
Короткая программа         55329
Элементы                    3860
Прыжки                       852
Интерпретация                106
Name: count, dtype: int64

Не более 3% данных по любительскому спорту. Отбросим их и будем работать только с данными "Произвольной программы" и "Короткой программы". Данный признак станет хорошим показателем для различия элементов.

In [97]:
if get_processed_data:
    data = data[
        data["segment_name"].isin(["Произвольная программа", "Короткая программа"])
    ]
data.shape

(166912, 31)

## Изучение поля "color"

In [98]:
data["color"].unique()

array(['green', 'lime'], dtype=object)

Цвета означают категорию спортсмена (спортсменов): мужчины, женщины, пары ... Точной информации не имеется. Примем, "как есть".

In [99]:
data["color"].value_counts()

color
green    120080
lime      46832
Name: count, dtype: int64

In [100]:
data["color"].isna().sum()

0

Распределение не очень приятно. Около 1:3. Пропусков не имеется. Используем признак в работе.

## Изучение поля "school_id"

In [101]:
data["school_id"].nunique()

239

In [102]:
data["school_id"].dtype

dtype('float64')

Школы фигурного катания представлены в большом количестве.

In [103]:
data["school_id"].isna().sum()

6875

Много пропусков, проверим, имеется ли значение-заглушка

In [104]:
sorted(data["school_id"].unique())

[4.0,
 5.0,
 6.0,
 7.0,
 15.0,
 16.0,
 18.0,
 19.0,
 20.0,
 21.0,
 23.0,
 24.0,
 25.0,
 26.0,
 27.0,
 29.0,
 30.0,
 33.0,
 34.0,
 35.0,
 37.0,
 38.0,
 40.0,
 49.0,
 50.0,
 57.0,
 61.0,
 62.0,
 63.0,
 64.0,
 65.0,
 67.0,
 72.0,
 75.0,
 76.0,
 82.0,
 84.0,
 90.0,
 91.0,
 92.0,
 94.0,
 96.0,
 97.0,
 100.0,
 103.0,
 105.0,
 110.0,
 111.0,
 117.0,
 118.0,
 122.0,
 123.0,
 126.0,
 127.0,
 128.0,
 133.0,
 135.0,
 137.0,
 139.0,
 146.0,
 150.0,
 151.0,
 152.0,
 155.0,
 156.0,
 164.0,
 167.0,
 169.0,
 172.0,
 173.0,
 174.0,
 176.0,
 177.0,
 182.0,
 184.0,
 186.0,
 187.0,
 188.0,
 189.0,
 192.0,
 194.0,
 195.0,
 196.0,
 198.0,
 199.0,
 200.0,
 201.0,
 203.0,
 205.0,
 206.0,
 208.0,
 209.0,
 212.0,
 215.0,
 223.0,
 224.0,
 228.0,
 229.0,
 232.0,
 235.0,
 238.0,
 243.0,
 244.0,
 246.0,
 249.0,
 251.0,
 252.0,
 nan,
 1.0,
 2.0,
 3.0,
 8.0,
 13.0,
 14.0,
 17.0,
 22.0,
 31.0,
 36.0,
 41.0,
 42.0,
 44.0,
 46.0,
 47.0,
 48.0,
 51.0,
 54.0,
 55.0,
 56.0,
 58.0,
 59.0,
 60.0,
 69.0,
 70.0,
 71.0,
 73.0,


Вероятнее всего, значение 99999 - заглушка. Используем ее для заполнения пропусков. А затем изменим на 0.

In [105]:
if get_processed_data:
    data[data["school_id"].isna()] = 99999
    data[data["school_id"] == 99999] = 0
data["school_id"].isna().sum()

0

Преобразуем тип данных в int16

In [106]:
if get_processed_data:
    data["school_id"] = data["school_id"].astype("int16")

## Изучение поля "unit_id"

In [107]:
data["unit_id"].nunique()

3033

In [108]:
data["unit_id"].dtype

dtype('int64')

In [109]:
data["unit_id"].isna().sum()

0

In [110]:
data["unit_id"].unique().min(), data["unit_id"].unique().max()

(0, 35024)

Каких-либо особенностей не замечено. Используем в работе.

## Изучение поля "tournament_id"

In [111]:
data["tournament_id"].nunique()

143

In [112]:
data["tournament_id"].dtype

dtype('int64')

In [113]:
data["tournament_id"].isna().sum()

0

In [114]:
data["tournament_id"].unique().min(), data["tournament_id"].unique().max()

(0, 7117)

Каких-либо особенностей не замечено. Используем в работе.

## Изучение поля "origin_id"

In [115]:
data["origin_id"].nunique()

3

In [116]:
data["origin_id"].unique()

array([2., 1., 0.])

In [117]:
data["origin_id"].dtype

dtype('float64')

In [118]:
data["origin_id"].isna().sum()

0

Преобразуем тип данных в int8

In [119]:
if get_processed_data:
    data["origin_id"] = data["origin_id"].astype("int8")

Каких-либо особенностей не замечено. Используем в работе.

## Изучение полей "date_start" и "date_end"

In [120]:
data["date_start"].isna().sum()

0

In [121]:
data["date_end"].isna().sum()

0

Имеются даты со значением 0.

In [122]:
data[data["date_start"] == 0].head(5)

,id,total_score_id,title,decrease,base_score,goe,avg_score,unit_id,tournament_id,base_score_total_scores,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,color,school_id,date_start,date_end,origin_id,sequences,cascade,title_nlp,cascade_nlp,multiply
1410,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0
1411,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0
1412,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0
1413,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0
1414,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0


Удалим такие строки.

In [123]:
if get_processed_data:
    data = data[data["date_start"] != 0]

Преобразуем поля к типу datetime

In [124]:
if get_processed_data:
    data["date_start"] = pd.to_datetime(data["date_start"])
    data["date_end"] = pd.to_datetime(data["date_end"])

In [125]:
data["date_start"].dt.year.unique()

array([2090, 2091, 2092, 2089])

Создадим некоторое количество новых полей на основании данных о сроках проведения соревнований.

In [126]:
if get_processed_data:
    # 1. Продолжительность соревнования
    data["tournament_duration"] = (data["date_end"] - data["date_start"]).dt.days

    # 2. Год начала и окончания соревнования
    data["start_year"] = data["date_start"].dt.year
    data["end_year"] = data["date_end"].dt.year

    # 3. Месяц начала и окончания соревнования
    data["start_month"] = data["date_start"].dt.month
    data["end_month"] = data["date_end"].dt.month

    # 4. День недели начала и окончания соревнования
    data["start_day_of_week"] = data["date_start"].dt.dayofweek
    data["end_day_of_week"] = data["date_end"].dt.dayofweek

    # 5. Признак "Выходной день"
    data["start_is_weekend"] = (data["date_start"].dt.dayofweek >= 5).astype(int)
    data["end_is_weekend"] = (data["date_end"].dt.dayofweek >= 5).astype(int)

    # 6. Признак "Время года"
    seasons = {
        1: "зима",
        2: "зима",
        3: "весна",
        4: "весна",
        5: "весна",
        6: "лето",
        7: "лето",
        8: "лето",
        9: "осень",
        10: "осень",
        11: "осень",
        12: "зима",
    }
    data["start_season"] = data["date_start"].dt.month.map(seasons)
    data["end_season"] = data["date_end"].dt.month.map(seasons)

Проверим результаты создания.

In [127]:
data["start_is_weekend"].unique()

array([0, 1])

In [128]:
data["end_is_weekend"].unique()

array([0, 1])

In [129]:
data["start_day_of_week"].unique()

array([2, 1, 6, 3, 5, 4, 0])

In [130]:
data["end_day_of_week"].unique()

array([4, 5, 3, 1, 6, 0, 2])

In [131]:
data["start_month"].unique()

array([11,  3,  1, 12, 10,  2,  5,  4,  9])

In [132]:
data["end_month"].unique()

array([12,  3,  1, 10, 11,  2,  5,  4,  9])

In [133]:
sum(data["start_season"] == data["end_season"]) == len(data)

False

In [134]:
if get_processed_data:
    display(sum(data["start_year"] == data["end_year"]) == len(data))

True

Год начала и год окончания совпадают, можно удалить. Заменим на год проведения

In [135]:
if get_processed_data:
    data["tournament_year"] = data["start_year"]
    data = data.drop(["start_year", "end_year"], axis=1)

In [136]:
sum(data["start_month"] == data["end_month"]) == len(data)

False

Оценим, сколько проводится соревнований в год

In [137]:
data.groupby(["tournament_year"])["tournament_id"].nunique()

tournament_year
2089     4
2090    32
2091    70
2092    36
Name: tournament_id, dtype: int64

Оценим, сколько спортсменов участвовали в каком количестве соревнований.

In [138]:
data.groupby(["unit_id"])["tournament_id"].nunique().sort_values(
    ascending=False
).value_counts()

tournament_id
1     851
2     539
3     408
4     318
5     211
6     171
7     118
8      93
9      82
10     54
11     47
12     29
13     25
14     25
15     17
17     10
16      9
18      8
19      7
20      7
23      2
21      1
Name: count, dtype: int64

Будем использовать только спортсменов, у которых имеется история соревнований - минимум 3 соревнования для расчета средних значений в окне временного ряда. Придется отказаться от включения в модель данных о 1390 спортсменах.

In [139]:
n_competitions = 3

In [140]:
competition_quantity_by_unit = data.groupby(["unit_id"])["tournament_id"].nunique()
ids_units = competition_quantity_by_unit[
    competition_quantity_by_unit < n_competitions
].index
len(ids_units)

1390

In [141]:
data.shape

(159498, 41)

In [142]:
data[data["unit_id"].isin(ids_units)].shape

(23017, 41)

В случае их исключения мы потеряем 14% данных. Определим новый столбец для того, чтобы пометить такие данные для исключения. Позднее примем решение.

In [143]:
if get_processed_data:
    data["units_with_experience"] = data["unit_id"].progress_apply(
        lambda x: 0 if x in ids_units else 1
    )

100%|██████████| 159498/159498 [00:00<00:00, 353653.75it/s]


## Изучение полей "overall_place", "overall_total_score", "overall_place_str"

In [144]:
data[["overall_place", "overall_total_score", "overall_place_str"]].isna().sum()

overall_place              0
overall_total_score        0
overall_place_str      81274
dtype: int64

In [145]:
data["overall_place_str"].nunique()

53

In [146]:
data["overall_place_str"].unique()

array([nan, '5', '8', '4', '2', '11', '13', '15', '12', '3', '7', '10',
       '18', '1', '6', '17', '14', '23', '9', '16', '19', '28', '25',
       '32', '20', '22', '24', 'NQD', '21', '27', 'WD', '26', '29', '30',
       'FNR', '31', '33', '34', '35', '37', '38', '41', '42', '43', '44',
       '45', '46', '47', '48', '36', '39', '40', '49', '50'], dtype=object)

In [147]:
data["overall_place_str"].value_counts()

overall_place_str
1      5555
2      5428
3      5253
4      5245
5      5114
6      4915
7      4556
8      4228
9      4049
10     3624
11     3303
12     3026
13     2617
14     2423
15     2225
16     2104
17     1855
18     1584
19     1346
20     1118
21     1105
22      994
NQD     867
23      848
24      749
25      659
26      456
27      381
28      357
WD      294
29      254
30      234
32      190
31      155
33      137
34      129
FNR      77
38       67
39       65
37       63
35       63
36       62
43       61
40       61
42       55
44       54
45       47
41       41
46       33
47       28
48       26
49       22
50       22
Name: count, dtype: int64

Обозначения 'NQD', 'WD', и 'FNR' используются в различных видах спорта для обозначения определенных состояний или результатов спортсменов в соревнованиях. Вот что они могут означать:

- NQD (Not Qualified):
    - Расшифровка: Не квалифицирован.
  - Описание: Спортсмен не прошел квалификационный этап соревнования и не может участвовать в следующих этапах.
- WD (Withdrawn):
  - Расшифровка: Снялся с соревнования.
  - Описание: Спортсмен снялся с соревнования по каким-либо причинам, например, из-за травмы или болезни.
- FNR (Final Not Reached):
  - Расшифровка: Финал не достигнут.
  - Описание: Спортсмен не смог пройти в финальный этап соревнования.

Эти обозначения помогают ясно и однозначно фиксировать результаты участия спортсменов в соревнованиях, включая случаи, когда спортсмены не доходят до финала или вынуждены прекратить участие по каким-либо причинам.

In [148]:
data["overall_place"].nunique()

51

In [149]:
data["overall_place"].unique()

array([ 4,  1,  5,  2,  3, 22, 18,  9, 15,  8, 14, 19, 10, 17, 16, 23, 24,
       12, 21,  6, 20,  7, 26, 13, 11, 25, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,  0, 50],
      dtype=int64)

In [150]:
data[data["overall_place_str"] == "NQD"].sample(2, random_state=1206)

,id,total_score_id,title,decrease,base_score,goe,avg_score,unit_id,tournament_id,base_score_total_scores,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,color,school_id,date_start,date_end,origin_id,sequences,cascade,title_nlp,cascade_nlp,multiply,tournament_duration,start_month,end_month,start_day_of_week,end_day_of_week,start_is_weekend,end_is_weekend,start_season,end_season,tournament_year,units_with_experience
167584,318823,459771,FSSp3,NaN,2.6,0.52,3.12,9687,7092,20.18,24.52,44.44,20.92,-1.0,43,31,Короткая программа,<< Пониженный прыжок x Надбавка за прыжки во в...,0,0.0,NQD,green,146,2092-03-29,2092-03-31,2,"[{'item': 'FSSp3', 'attr': [], 'fail': None}]",FSSp3,Flying Sit Spin (level 3),Flying Sit Spin (level 3),0,2,3,3,5,0,1,0,весна,весна,2092,0
117899,157973,442245,StSq1,NaN,1.8,0.24,2.04,2163,5459,20.60,22.88,41.31,19.43,-1.0,43,35,Короткая программа,q Прыжок приземлён в четверть < Недокрученный ...,0,0.0,NQD,green,27,2092-01-10,2092-01-12,1,"[{'item': 'StSq1', 'attr': [], 'fail': None}]",StSq1,Step Sequence (level 1),Step Sequence (level 1),0,2,1,1,3,5,0,1,зима,зима,2092,1


In [151]:
data[data["overall_place_str"] == "NQD"]["segment_name"].unique()

array(['Короткая программа'], dtype=object)

Не проходят квалификационный отбор только в "Короткой программе".

In [152]:
data[data["overall_place_str"] == "WD"].sample(2, random_state=1206)

,id,total_score_id,title,decrease,base_score,goe,avg_score,unit_id,tournament_id,base_score_total_scores,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,color,school_id,date_start,date_end,origin_id,sequences,cascade,title_nlp,cascade_nlp,multiply,tournament_duration,start_month,end_month,start_day_of_week,end_day_of_week,start_is_weekend,end_is_weekend,start_season,end_season,tournament_year,units_with_experience
120685,161175,442672,CCoSp4,NaN,3.5,0.70,4.20,5095,4844,40.01,41.08,86.38,45.3,0.0,7,2,Короткая программа,x Надбавка за прыжки во второй половине програ...,0,0.0,WD,lime,62,2091-12-25,2091-12-26,1,"[{'item': 'CCoSp4', 'attr': [], 'fail': None}]",CCoSp4,Change Foot Combination Spin (level 4),Change Foot Combination Spin (level 4),0,1,12,12,1,2,0,0,зима,зима,2091,1
90911,194558,446157,CCoSp4,NaN,3.5,1.28,4.78,1343,6676,32.65,33.53,71.33,37.8,0.0,23,13,Короткая программа,x Надбавка за прыжки во второй половине програ...,0,0.0,WD,lime,62,2092-02-28,2092-03-01,1,"[{'item': 'CCoSp4', 'attr': [], 'fail': None}]",CCoSp4,Change Foot Combination Spin (level 4),Change Foot Combination Spin (level 4),0,2,2,3,3,5,0,1,зима,весна,2092,1


In [153]:
data[data["overall_place_str"] == "WD"]["segment_name"].unique()

array(['Короткая программа', 'Произвольная программа'], dtype=object)

Снимаются с соревнований и в "Короткой программе" и в "Произвольной программе".

In [154]:
data[data["overall_place_str"] == "FNR"].sample(2, random_state=1206)

,id,total_score_id,title,decrease,base_score,goe,avg_score,unit_id,tournament_id,base_score_total_scores,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,color,school_id,date_start,date_end,origin_id,sequences,cascade,title_nlp,cascade_nlp,multiply,tournament_duration,start_month,end_month,start_day_of_week,end_day_of_week,start_is_weekend,end_is_weekend,start_season,end_season,tournament_year,units_with_experience
158469,343546,462309,3Lo,NaN,4.9,0.65,5.55,339,7116,45.17,64.68,114.76,50.08,0.0,7,2,Произвольная программа,* Недопустимый элемент x Надбавка за прыжки во...,0,0.0,FNR,lime,62,2091-10-24,2091-10-27,0,"[{'item': '3Lo', 'attr': [], 'fail': None}]",3Lo,Triple Loop,Triple Loop,0,3,10,10,2,5,0,1,осень,осень,2091,1
158480,343557,462310,2A,NaN,3.3,0.00,3.30,212,7116,37.13,45.53,77.16,34.63,-3.0,2,6,Произвольная программа,q Прыжок приземлён в четверть < Недокрученный ...,0,0.0,FNR,lime,62,2091-10-24,2091-10-27,0,"[{'item': '2A', 'attr': [], 'fail': None}]",2A,Double Axel,Double Axel,0,3,10,10,2,5,0,1,осень,осень,2091,1


In [155]:
data[data["overall_place_str"] == "FNR"]["segment_name"].unique()

array(['Короткая программа', 'Произвольная программа'], dtype=object)

Не доходят до финала и в "Короткой программе" и в "Произвольной программе".

In [156]:
if get_processed_data:
    data[data["overall_place_str"].isna()].sample(2, random_state=1206)

In [157]:
if get_processed_data:
    data[data["overall_place_str"].isna()]["segment_name"].unique()

Заполним пропуски *строкой*, соответствующей столбцу 'overall_place'

In [158]:
if get_processed_data:
    data["overall_place_str"] = data.apply(
        lambda row: str(row["overall_place"])
        if pd.isna(row["overall_place_str"])
        else row["overall_place_str"],
        axis=1,
    )

In [159]:
data["overall_place_str"].unique()

array(['4', '1', '5', '2', '3', '22', '18', '9', '15', '8', '14', '19',
       '10', '17', '16', '23', '24', '12', '21', '6', '20', '7', '26',
       '13', '11', '25', '27', '28', '29', '30', '31', '32', '33', '34',
       '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45',
       '46', '47', '48', '49', '0', 'NQD', 'WD', 'FNR', '50'],
      dtype=object)

Изучим, что означает место 0

In [160]:
data[(data["overall_place_str"] == "0")].sample(10, random_state=RANDOM_STATE)

,id,total_score_id,title,decrease,base_score,goe,avg_score,unit_id,tournament_id,base_score_total_scores,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,color,school_id,date_start,date_end,origin_id,sequences,cascade,title_nlp,cascade_nlp,multiply,tournament_duration,start_month,end_month,start_day_of_week,end_day_of_week,start_is_weekend,end_is_weekend,start_season,end_season,tournament_year,units_with_experience
171536,351668,463359,2S+1A+SEQ,NaN,2.4,0.13,2.53,158,7117,15.74,23.12,39.43,16.31,0.0,9,6,Произвольная программа,< Недокрученный прыжок,0,0.0,0,green,198,2092-04-29,2092-05-03,0,"[{'item': '2S', 'attr': [], 'fail': None}, {'i...",2S+1A+SEQ,Double Salchow and Single Axel and Sequence,Double Salchow and Single Axel and Sequence,0,4,4,5,1,5,0,1,весна,весна,2092,1
172109,351968,463401,CCoSp1,NaN,2.0,0.27,2.27,2317,7117,15.22,22.67,38.71,16.04,0.0,3,1,Произвольная программа,< Недокрученный прыжок ! Неясное ребро на толч...,0,0.0,0,lime,198,2092-04-29,2092-05-03,0,"[{'item': 'CCoSp1', 'attr': [], 'fail': None}]",CCoSp1,Change Foot Combination Spin (level 1),Change Foot Combination Spin (level 1),0,4,4,5,1,5,0,1,весна,весна,2092,1
156996,291127,456057,2Lz+2Lo+2Lo,NaN,0.0,-0.21,5.84,744,7109,35.71,47.72,77.06,32.34,-3.0,3,12,Произвольная программа,< Недокрученный прыжок << Пониженный прыжок ! ...,0,0.0,0,lime,216,2092-04-08,2092-04-10,2,"[{'item': '2Lz', 'attr': [], 'fail': None}, {'...",2Lz+2Lo+2Lo,Double Lutz and Double Loop and Double Loop,Double Lutz and Double Loop and Double Loop,1,2,4,4,1,3,0,0,весна,весна,2092,1
167092,351360,463315,1Lz,NaN,0.6,0.06,0.66,9997,7117,7.84,18.53,25.69,7.66,-0.5,9,5,Произвольная программа,q Прыжок приземлён в четверть < Недокрученный ...,0,0.0,0,green,198,2092-04-29,2092-05-03,0,"[{'item': '1Lz', 'attr': [], 'fail': None}]",1Lz,Single Lutz,Single Lutz,0,4,4,5,1,5,0,1,весна,весна,2092,1
138541,353255,463572,2F<<,<<,0.0,-0.15,0.40,4549,7117,16.09,24.48,39.86,15.38,0.0,2,17,Произвольная программа,< Недокрученный прыжок << Пониженный прыжок x ...,0,0.0,0,green,256,2092-04-29,2092-05-03,0,"[{'item': '2F', 'attr': ['<<'], 'fail': None}]",2F,Double Flip downgraded jump / downgraded twist...,Double Flip,1,4,4,5,1,5,0,1,весна,весна,2092,1
96064,197041,446445,FCCoSp4,NaN,3.5,0.70,4.20,1258,6160,46.73,42.28,79.61,40.33,-3.0,4,14,Произвольная программа,"Представление@@@2.67@@@5,75@@@5,00@@@4,75@@@4,...",0,0.0,0,green,180,2092-01-29,2092-01-30,2,"[{'item': 'FCCoSp4', 'attr': [], 'fail': None}]",FCCoSp4,Flying Change Foot Comb. Spin (level 4),Flying Change Foot Comb. Spin (level 4),0,1,1,1,1,2,0,0,зима,зима,2092,1
162214,307362,458271,CSSp3,NaN,2.6,0.35,2.95,1913,7110,16.40,19.74,36.05,17.31,-1.0,9,12,Произвольная программа,F Падение в элементе,0,0.0,0,green,274,2092-04-09,2092-04-12,1,"[{'item': 'CSSp3', 'attr': [], 'fail': None}]",CSSp3,Change Foot Sit Spin (level 3),Change Foot Sit Spin (level 3),0,3,4,4,2,5,0,1,весна,весна,2092,1
138579,353286,463577,StSqB,NaN,1.5,0.15,1.65,2437,7117,15.29,14.72,29.47,14.75,0.0,5,5,Короткая программа,< Недокрученный прыжок << Пониженный прыжок x ...,0,0.0,0,lime,208,2092-04-29,2092-05-03,0,"[{'item': 'StSqB', 'attr': [], 'fail': None}]",StSqB,Step Sequence (level B),Step Sequence (level B),0,4,4,5,1,5,0,1,весна,весна,2092,1
157565,291696,456119,3Lz,NaN,0.0,0.59,7.08,546,7109,58.22,61.27,116.76,57.49,-2.0,4,8,Произвольная программа,<< Пониженный прыжок x Надбавка за прыжки во в...,0,0.0,0,lime,194,2092-04-08,2092-04-10,2,"[{'item': '3Lz', 'attr': [], 'fail': None}]",3Lz,Triple Lutz,Triple Lutz,1,2,4,4,1,3,0,0,весна,весна,2092,1
138870,353536,463608,2F+2Lo,NaN,0.0,0.18,4.03,132,7117,36.34,41.17,76.93,35.76,0.0,15,6,Произвольная программа,< Недокрученный прыжок x Надбавка за прыжки во...,0,0.0,0,green,208,2092-04-29,2092-05-03,0,"[{'item': '2F', 'attr': [], 'fail': None}, {'i...",2F+2Lo,Double Flip and Double Loop,Double Flip and Double Loop,1,4,4,5,1,5,0,1,весна,в

Сложно предположить, почему выполненные элементы не были засчитаны в соревновании. Увидеть какой либо зависимости не удается. Минимальное место в соревнования в представленных данных - 50. Будем считать, что данные спортсмены не попали в ТОП50 и для них установлено значение занятого места, равное 0.

## Изучение поля "info"

In [161]:
data["info"].nunique()

1150

In [162]:
data[["info"]].sample(5, random_state=RANDOM_STATE)

,info
1367,q Прыжок приземлён в четверть REP Повторение п...
54625,x Надбавка за прыжки во второй половине програ...
97211,q Прыжок приземлён в четверть ! Неясное ребро ...
13662,< Недокрученный прыжок x Надбавка за прыжки во...
92162,q Прыжок приземлён в четверть x Надбавка за пр...


Надбавки, указанные в столбце мы уже учли. Другие сведения, например, штрафы учтены в других столбцах, в частности, в 'title', 'decrease' и 'decrease_score'.

Не будем работать с указанным столбцом в настоящей работе. Заполним пропуски пустой строкой, чтобы не раздражали.

In [163]:
if get_processed_data:
    data["info"] = data["info"].fillna("")

## Изучение полей "total_score", "overall_total_score"

In [164]:
data[["total_score", "overall_total_score"]].isna().sum()

total_score            0
overall_total_score    0
dtype: int64

In [165]:
data[["total_score", "overall_total_score", "tournament_id", "unit_id"]].sample(
    10, random_state=RANDOM_STATE
)

,total_score,overall_total_score,tournament_id,unit_id
1367,96.99,154.78,2,79
54625,109.93,167.74,72,361
97211,117.54,183.15,2616,1504
13662,123.76,189.36,17,720
92162,111.43,170.11,6150,9685
21711,47.43,154.32,23,998
40876,116.02,177.15,45,723
153064,24.90,24.90,7098,9837
164249,77.43,122.68,7111,30
75721,102.14,192.50,6699,989


Некоторые значения совпадают, некоторые значения "overall_total_score" равны 0 при наличии 'total_score'. Изучим поближе.

In [166]:
data[(data["unit_id"] == 2023) & (data["tournament_id"] == 7117)][
    [
        "title",
        "base_score",
        "goe",
        "base_score_total_scores",
        "components_score",
        "segment_name",
        "total_score",
        "elements_score",
        "decreasings_score",
        "overall_total_score",
        "overall_place",
        "overall_place_str",
    ]
]

,title,base_score,goe,base_score_total_scores,components_score,segment_name,total_score,elements_score,decreasings_score,overall_total_score,overall_place,overall_place_str
75776,1A,1.10,0.11,19.06,23.23,Произвольная программа,42.52,19.29,0.0,0.0,0,0
75777,2F+2Lo,3.50,0.00,19.06,23.23,Произвольная программа,42.52,19.29,0.0,0.0,0,0
75778,2Lz+2Lo<,3.46,-0.42,19.06,23.23,Произвольная программа,42.52,19.29,0.0,0.0,0,0
75779,FSSp3,2.60,0.26,19.06,23.23,Произвольная программа,42.52,19.29,0.0,0.0,0,0
75780,StSqB,1.50,0.15,19.06,23.23,Произвольная программа,42.52,19.29,0.0,0.0,0,0
75781,2Fq,1.80,-0.24,19.06,23.23,Произвольная программа,42.52,19.29,0.0,0.0,0,0
75782,2Lz,2.10,0.07,19.06,23.23,Произвольная программа,42.52,19.29,0.0,0.0,0,0
75783,CCoSp3,3.00,0.30,19.06,23.23,Произвольная программа,42.52,19.29,0.0,0.0,0,0


Спортсмен не участвовал в "Короткой программе". В этой связи, возможно у него не имеется данных за всё соревнование. Будем считать, что спортсмен за соревнование занял условно "последнее место". Это повлияет на агрегацию данных - среднее занятое место будет ниже, что косвенно означает, что спортсмен "ниже" уровнем.

Оценим, сколько таких записей.

In [167]:
data[data["overall_total_score"] == 0].shape

(4742, 42)

Не много. Предлагается их не удалять.

In [168]:
data[(data["unit_id"] == 2019) & (data["tournament_id"] == 222)][
    [
        "title",
        "base_score",
        "goe",
        "base_score_total_scores",
        "components_score",
        "segment_name",
        "total_score",
        "elements_score",
        "decreasings_score",
        "overall_total_score",
        "overall_place",
        "overall_place_str",
    ]
]

,title,base_score,goe,base_score_total_scores,components_score,segment_name,total_score,elements_score,decreasings_score,overall_total_score,overall_place,overall_place_str
96852,3Lz,5.90,1.18,30.13,29.65,Короткая программа,61.94,32.29,0.0,174.18,21,21
96853,3Lo+3T,9.10,-0.98,30.13,29.65,Короткая программа,61.94,32.29,0.0,174.18,21,21
96854,CCSp3,2.80,0.37,30.13,29.65,Короткая программа,61.94,32.29,0.0,174.18,21,21
96855,StSq2,2.60,0.26,30.13,29.65,Короткая программа,61.94,32.29,0.0,174.18,21,21
96856,2A,0.00,0.55,30.13,29.65,Короткая программа,61.94,32.29,0.0,174.18,21,21
96857,CCoSp4,3.50,0.35,30.13,29.65,Короткая программа,61.94,32.29,0.0,174.18,21,21
96858,FSSp3,2.60,0.43,30.13,29.65,Короткая программа,61.94,32.29,0.0,174.18,21,21
99963,3Lz<,4.72,-2.36,53.90,60.24,Произвольная программа,112.24,53.00,-1.0,174.18,21,21
99964,3F,5.30,1.06,53.90,60.24,Произвольная программа,112.24,53.00,-1.0,174.18,21,21
99965,3S,4.30,0.00,53.90,60.24,Произвольная программа,112.24,53.00,-1.0,174.18,21,21


Спортсмен на соревновании участвовал в обоих программах. В связи с этим 'overall_total_score' равны сумме 'total_score' за каждую программу.

При этом, попробуем понять, по какой причине у спортсмена появился один балл в 'decreasings_score'.

In [169]:
data[(data["unit_id"] == 2019) & (data["tournament_id"] == 222)][["info", "decrease"]]

,info,decrease
96852,x Надбавка за прыжки во второй половине програ...,NaN
96853,x Надбавка за прыжки во второй половине програ...,NaN
96854,x Надбавка за прыжки во второй половине програ...,NaN
96855,x Надбавка за прыжки во второй половине програ...,NaN
96856,x Надбавка за прыжки во второй половине програ...,NaN
96857,x Надбавка за прыжки во второй половине програ...,NaN
96858,x Надбавка за прыжки во второй половине програ...,NaN
99963,< Недокрученный прыжок x Надбавка за прыжки во...,F
99964,< Недокрученный прыжок x Надбавка за прыжки во...,NaN
99965,< Недокрученный прыжок x Надбавка за прыжки во...,NaN


In [170]:
data[(data["unit_id"] == 2019) & (data["tournament_id"] == 222)]["info"].unique()

array(['x Надбавка за прыжки во второй половине программы (10%)',
       '< Недокрученный прыжок x Надбавка за прыжки во второй половине программы (10%) F Падение в элементе'],
      dtype=object)

Можно предположить, что баллы снижены за падение в элементе. Причина этого снижения указана в 'decrease'.

In [171]:
data[(data["unit_id"] == 2019) & (data["tournament_id"] == 222)][
    ["title", "base_score", "goe", "avg_score", "decrease"]
]

,title,base_score,goe,avg_score,decrease
96852,3Lz,5.90,1.18,7.08,NaN
96853,3Lo+3T,9.10,-0.98,8.12,NaN
96854,CCSp3,2.80,0.37,3.17,NaN
96855,StSq2,2.60,0.26,2.86,NaN
96856,2A,0.00,0.55,4.18,NaN
96857,CCoSp4,3.50,0.35,3.85,NaN
96858,FSSp3,2.60,0.43,3.03,NaN
99963,3Lz<,4.72,-2.36,2.36,F
99964,3F,5.30,1.06,6.36,NaN
99965,3S,4.30,0.00,4.30,NaN


## Изучение полей "decrease" и "decreasing_score"

В поле 'decrease' очень много пропусков. Изучим необходимость данного поля.

In [172]:
data["decrease"].unique()

array([nan, 'q', '<', '!', '<<', 'nS', 'F', '*', 'e', 'nU', 'nC', '<<*',
       '<*', 'nS*', '!F', 'B', 'F*', '!*', 'q*', 'nF', 'nB', 'B.', 'f*',
       'В', '<F', 'qF', 'e*', 'b', '!<', 'f', 'FnU', 'FF'], dtype=object)

Существуют ли записи с ошибками, указанными в 'title', но отсутствующими в 'decrease'?

In [173]:
if get_processed_data:
    data[(data["title"].str.contains("<<")) & (data["decrease"].isna())].sample(
        1, random_state=RANDOM_STATE
    )

Изучим совпадающие записи.

In [174]:
data[(data["title"].str.contains("<<")) & (data["decrease"] == "<<")].sample(
    1, random_state=RANDOM_STATE
)

,id,total_score_id,title,decrease,base_score,goe,avg_score,unit_id,tournament_id,base_score_total_scores,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,color,school_id,date_start,date_end,origin_id,sequences,cascade,title_nlp,cascade_nlp,multiply,tournament_duration,start_month,end_month,start_day_of_week,end_day_of_week,start_is_weekend,end_is_weekend,start_season,end_season,tournament_year,units_with_experience
171028,322202,460174,2S<<,<<,0.4,-0.2,0.2,10078,6866,8.8,13.73,22.31,8.58,0.0,13,11,Произвольная программа,<< Пониженный прыжок,11,22.31,11,green,253,2092-02-19,2092-02-21,0,"[{'item': '2S', 'attr': ['<<'], 'fail': None}]",2S,Double Salchow downgraded jump / downgraded tw...,Double Salchow,0,2,2,2,1,3,0,0,зима,зима,2092,0


Изучим записи с ошибками, не отраженными в 'title'.

In [175]:
data[data["decrease"] == "nU"].sample(1, random_state=RANDOM_STATE)

,id,total_score_id,title,decrease,base_score,goe,avg_score,unit_id,tournament_id,base_score_total_scores,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,color,school_id,date_start,date_end,origin_id,sequences,cascade,title_nlp,cascade_nlp,multiply,tournament_duration,start_month,end_month,start_day_of_week,end_day_of_week,start_is_weekend,end_is_weekend,start_season,end_season,tournament_year,units_with_experience
156331,290167,455976,CCoSp2V,nU,1.88,-0.25,1.63,2027,6852,15.31,19.8,34.32,15.02,-0.5,13,9,Произвольная программа,F Падение в элементе nU Нет базовой позиции 'с...,9,34.32,9,green,198,2091-11-29,2091-12-01,0,"[{'item': 'CCoSp2', 'attr': ['V'], 'fail': None}]",CCoSp2,Change Foot Combination Spin (level 2) reduce...,Change Foot Combination Spin (level 2),0,2,11,12,3,5,0,1,осень,зима,2091,1


In [176]:
data[data["decrease"] == "FF"].sample(1, random_state=RANDOM_STATE)

,id,total_score_id,title,decrease,base_score,goe,avg_score,unit_id,tournament_id,base_score_total_scores,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,color,school_id,date_start,date_end,origin_id,sequences,cascade,title_nlp,cascade_nlp,multiply,tournament_duration,start_month,end_month,start_day_of_week,end_day_of_week,start_is_weekend,end_is_weekend,start_season,end_season,tournament_year,units_with_experience
125024,212832,448448,ChSq1,FF,3.0,-2.5,0.5,3179,6845,14.3,20.11,28.15,10.04,-2.0,5,11,Произвольная программа,< Недокрученный прыжок << Пониженный прыжок F ...,11,28.15,11,green,38,2091-10-25,2091-10-27,0,"[{'item': 'ChSq1', 'attr': [], 'fail': None}]",ChSq1,Choreo Sequence,Choreo Sequence,0,2,10,10,3,5,0,1,осень,осень,2091,1


Указанной записи соответствует следующее содержание столбца 'info'.

In [177]:
data.loc[125024, "info"]

"< Недокрученный прыжок << Пониженный прыжок F Падение в элементе nC Нет базовой позиции 'либела'"

Можно отметить, что падение в элементе оказывает значительное влияние на итоговую оценку спортсмена. В системе судейства Международного союза конькобежцев (ISU), которая используется на международных соревнованиях, баллы снимаются следующим образом:

- Снижение базовой стоимости элемента:
  - Падение приводит к снижению базовой стоимости элемента, так как судьи технической панели могут понизить уровень сложности элемента или снизить оценку за недокруты и другие ошибки.
- Вычет GOE (Grade of Execution):
  - Каждый элемент оценивается по шкале GOE от -5 до +5. Падение обычно приводит к максимально негативному значению GOE (-5), что существенно снижает оценку элемента.
- Штраф за падение:
  - **За каждое падение применяется фиксированный штраф в размере 1 балла**, который вычитается из итоговой суммы баллов. Этот штраф применяется независимо от других вычетов и касается как короткой программы, так и произвольной программы.

Пример:
Предположим, фигурист выполняет тройной прыжок с базовой стоимостью 8 баллов. Из-за падения:

- Базовая стоимость элемента может быть пересмотрена, если падение сопровождалось недокрутом или другим нарушением.
- Судьи GOE могут снизить оценку до -5, что уменьшит итоговые баллы за элемент.
- Применяется штраф в размере 1 балла за само падение.

Таким образом, падение негативно влияет на общую оценку за программу, и спортсмены стараются избегать падений, чтобы сохранить высокие баллы.

Однако, проверим, откуда могут браться штрафы в пол-балла. Очень странно выглядят данные значения.

In [178]:
data[data["decreasings_score"] == -0.5]["segment_name"].value_counts()

segment_name
Произвольная программа    7552
Короткая программа        1157
Name: count, dtype: int64

In [179]:
data[data["decreasings_score"] == -1.0]["segment_name"].value_counts()

segment_name
Произвольная программа    11912
Короткая программа         5845
Name: count, dtype: int64

От программы не зависит. Имеется предположение, что это зависит от системы судейства, о которой у нас не имеется полной информации. Примем, как есть. В гипотетической системе судейства, которую можно представить для клубных соревнований, штрафы за падения могут быть такими:
- Падение на простом элементе: -0.5 балла
- Падение на сложном элементе: -1 балл
- Падение с некорректным выполнением элемента: -1.5 балла

Отобразим количество штрафов (ошибок), которые представлены и в столбце 'decrease' и в столбце 'title'

In [180]:
for error in data["decrease"].unique()[1:]:
    num = len(data[data["title"].str.contains(error, regex=False)])
    print(f"Ошибка {error} встречается в {num} элементах")

Ошибка q встречается в 27290 элементах
Ошибка < встречается в 20168 элементах
Ошибка ! встречается в 5010 элементах
Ошибка << встречается в 6845 элементах
Ошибка nS встречается в 0 элементах
Ошибка F встречается в 34244 элементах
Ошибка * встречается в 1507 элементах
Ошибка e встречается в 1080 элементах
Ошибка nU встречается в 0 элементах
Ошибка nC встречается в 0 элементах
Ошибка <<* встречается в 77 элементах
Ошибка <* встречается в 163 элементах
Ошибка nS* встречается в 0 элементах
Ошибка !F встречается в 0 элементах
Ошибка B встречается в 5377 элементах
Ошибка F* встречается в 62 элементах
Ошибка !* встречается в 23 элементах
Ошибка q* встречается в 37 элементах
Ошибка nF встречается в 0 элементах
Ошибка nB встречается в 0 элементах
Ошибка B. встречается в 0 элементах
Ошибка f* встречается в 0 элементах
Ошибка В встречается в 0 элементах
Ошибка <F встречается в 0 элементах
Ошибка qF встречается в 0 элементах
Ошибка e* встречается в 6 элементах
Ошибка b встречается в 0 элементах
Ош

- Ошибки: q < ! << e B !< - отметки о недостатках исполнения элементов. Активно встречаются в поле 'title'

- Ошибки: F !F <F qF - отметки о возможном падении. Не указаны в 'title' именно отметки F. Другие отметки - см. выше.

- Ошибки: B. f* В (вторая) b f FnU FF - неизвестные отметки. Предполагается, что это ошибки сбора данных.

- Ошибки: <<* * <* F* !* q* e* - отметки о недостатках исполнения элементов с отметкой о невозможности засчитать элемент, как выполненный (*). Все отметки представлены в поле 'title'

- Ошибки: nS nU nC nB nF - отметки, используемые для указания различных видов ошибок или недочетов, которые могут возникнуть при выполнении прыжков и других элементов. Эти отметки влияют на подсчет очков, поскольку каждая из них указывает на снижение базовой стоимости элемента или его оценку. Проверим, каким образом они влияют на расчеты в нашем дата-сете. Не встречаются в 'title'.

- Ошибки: nS* - аналогичные отметки, но с отметкой о невозможности засчитать элемент.

In [181]:
data[data["decrease"] == "nS"].sample(3, random_state=RANDOM_STATE)

,id,total_score_id,title,decrease,base_score,goe,avg_score,unit_id,tournament_id,base_score_total_scores,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,color,school_id,date_start,date_end,origin_id,sequences,cascade,title_nlp,cascade_nlp,multiply,tournament_duration,start_month,end_month,start_day_of_week,end_day_of_week,start_is_weekend,end_is_weekend,start_season,end_season,tournament_year,units_with_experience
27804,32699,7207,FCCoSp3V,nS,2.25,0.36,2.61,63,30,52.30,52.07,93.46,45.39,0.0,14,14,Произвольная программа,q Прыжок приземлён в четверть ! Неясное ребро ...,11,149.00,11,green,252,2091-02-10,2091-02-11,2,"[{'item': 'FCCoSp3', 'attr': ['V'], 'fail': No...",FCCoSp3,Flying Change Foot Comb. Spin (level 3) reduc...,Flying Change Foot Comb. Spin (level 3),0,1,2,2,5,6,1,1,зима,зима,2091,1
163754,308904,458450,FCCoSp2V,nS,1.88,0.34,2.22,377,7111,54.39,49.66,110.52,60.86,0.0,4,3,Произвольная программа,! Неясное ребро на толчке F/Lz x Надбавка за п...,6,161.51,6,green,184,2092-04-11,2092-04-13,2,"[{'item': 'FCCoSp2', 'attr': ['V'], 'fail': No...",FCCoSp2,Flying Change Foot Comb. Spin (level 2) reduc...,Flying Change Foot Comb. Spin (level 2),0,2,4,4,4,6,0,1,весна,весна,2092,1
39826,46874,8949,CCoSp3V,nS,2.25,0.68,2.93,1337,44,58.74,57.14,121.33,64.19,0.0,16,16,Произвольная программа,q Прыжок приземлён в четверть ! Неясное ребро ...,14,186.38,14,green,62,2091-03-30,2091-04-03,2,"[{'item': 'CCoSp3', 'attr': ['V'], 'fail': None}]",CCoSp3,Change Foot Combination Spin (level 3) reduce...,Change Foot Combination Spin (level 3),0,4,3,4,4,1,0,0,весна,весна,2091,1


In [182]:
data[data["decrease"] == "nU"].sample(3, random_state=RANDOM_STATE)

,id,total_score_id,title,decrease,base_score,goe,avg_score,unit_id,tournament_id,base_score_total_scores,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,color,school_id,date_start,date_end,origin_id,sequences,cascade,title_nlp,cascade_nlp,multiply,tournament_duration,start_month,end_month,start_day_of_week,end_day_of_week,start_is_weekend,end_is_weekend,start_season,end_season,tournament_year,units_with_experience
156331,290167,455976,CCoSp2V,nU,1.88,-0.25,1.63,2027,6852,15.31,19.80,34.32,15.02,-0.5,13,9,Произвольная программа,F Падение в элементе nU Нет базовой позиции 'с...,9,34.32,9,green,198,2091-11-29,2091-12-01,0,"[{'item': 'CCoSp2', 'attr': ['V'], 'fail': None}]",CCoSp2,Change Foot Combination Spin (level 2) reduce...,Change Foot Combination Spin (level 2),0,2,11,12,3,5,0,1,осень,зима,2091,1
51232,59787,11120,CCoSp2V,nU,1.88,-0.19,1.69,204,68,15.42,18.14,33.14,15.00,0.0,4,7,Произвольная программа,< Недокрученный прыжок nU Нет базовой позиции ...,7,33.14,7,lime,256,2091-05-16,2091-05-19,1,"[{'item': 'CCoSp2', 'attr': ['V'], 'fail': None}]",CCoSp2,Change Foot Combination Spin (level 2) reduce...,Change Foot Combination Spin (level 2),0,3,5,5,2,5,0,1,весна,весна,2091,1
129228,217111,449020,CCoSp1V,nU,1.50,0.05,1.55,2085,6848,11.24,14.45,25.90,11.45,0.0,13,13,Короткая программа,! Неясное ребро на толчке F/Lz x Надбавка за п...,11,78.28,11,green,64,2091-11-11,2091-11-15,0,"[{'item': 'CCoSp1', 'attr': ['V'], 'fail': None}]",CCoSp1,Change Foot Combination Spin (level 1) reduce...,Change Foot Combination Spin (level 1),0,4,11,11,6,3,1,0,осень,осень,2091,0


In [183]:
data[data["total_score_id"] == 455976]

,id,total_score_id,title,decrease,base_score,goe,avg_score,unit_id,tournament_id,base_score_total_scores,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,color,school_id,date_start,date_end,origin_id,sequences,cascade,title_nlp,cascade_nlp,multiply,tournament_duration,start_month,end_month,start_day_of_week,end_day_of_week,start_is_weekend,end_is_weekend,start_season,end_season,tournament_year,units_with_experience
156325,290161,455976,2F+1A+SEQ,NaN,2.90,0.18,3.08,2027,6852,15.31,19.8,34.32,15.02,-0.5,13,9,Произвольная программа,F Падение в элементе nU Нет базовой позиции 'с...,9,34.32,9,green,198,2091-11-29,2091-12-01,0,"[{'item': '2F', 'attr': [], 'fail': None}, {'i...",2F+1A+SEQ,Double Flip and Single Axel and Sequence,Double Flip and Single Axel and Sequence,0,2,11,12,3,5,0,1,осень,зима,2091,1
156326,290162,455976,2Lo+1A+SEQ,NaN,2.80,0.11,2.91,2027,6852,15.31,19.8,34.32,15.02,-0.5,13,9,Произвольная программа,F Падение в элементе nU Нет базовой позиции 'с...,9,34.32,9,green,198,2091-11-29,2091-12-01,0,"[{'item': '2Lo', 'attr': [], 'fail': None}, {'...",2Lo+1A+SEQ,Double Loop and Single Axel and Sequence,Double Loop and Single Axel and Sequence,0,2,11,12,3,5,0,1,осень,зима,2091,1
156327,290163,455976,2Lo,NaN,1.70,0.17,1.87,2027,6852,15.31,19.8,34.32,15.02,-0.5,13,9,Произвольная программа,F Падение в элементе nU Нет базовой позиции 'с...,9,34.32,9,green,198,2091-11-29,2091-12-01,0,"[{'item': '2Lo', 'attr': [], 'fail': None}]",2Lo,Double Loop,Double Loop,0,2,11,12,3,5,0,1,осень,зима,2091,1
156328,290164,455976,CSSp2V,F,1.73,-0.87,0.86,2027,6852,15.31,19.8,34.32,15.02,-0.5,13,9,Произвольная программа,F Падение в элементе nU Нет базовой позиции 'с...,9,34.32,9,green,198,2091-11-29,2091-12-01,0,"[{'item': 'CSSp2', 'attr': ['V'], 'fail': None}]",CSSp2,Change Foot Sit Spin (level 2) reduced value ...,Change Foot Sit Spin (level 2),0,2,11,12,3,5,0,1,осень,зима,2091,1
156329,290165,455976,ChSq1,NaN,3.00,0.33,3.33,2027,6852,15.31,19.8,34.32,15.02,-0.5,13,9,Произвольная программа,F Падение в элементе nU Нет базовой позиции 'с...,9,34.32,9,green,198,2091-11-29,2091-12-01,0,"[{'item': 'ChSq1', 'attr': [], 'fail': None}]",ChSq1,Choreo Sequence,Choreo Sequence,0,2,11,12,3,5,0,1,осень,зима,2091,1
156330,290166,455976,2S,NaN,1.30,0.04,1.34,2027,6852,15.31,19.8,34.32,15.02,-0.5,13,9,Произвольная программа,F Падение в элементе nU Нет базовой позиции 'с...,9,34.32,9,green,198,2091-11-29,2091-12-01,0,"[{'item': '2S', 'attr': [], 'fail': None}]",2S,Double Salchow,Double Salchow,0,2,11,12,3,5,0,1,осень,зима,2091,1
156331,290167,455976,CCoSp2V,nU,1.88,-0.25,1.63,2027,6852,15.31,19.8,34.32,15.02,-0.5,13,9,Произвольная программа,F Падение в элементе nU Нет базовой позиции 'с...,9,34.32,9,green,198,2091-11-29,2091-12-01,0,"[{'item': 'CCoSp2', 'attr': ['V'], 'fail': None}]",CCoSp2,Change Foot Combination Spin (level 2) reduce...,Change Foot Combination Spin (level 2),0,2,11,12,3,5,0,1,осень,зима,2091,1


In [184]:
data[data["decrease"] == "nC"].sample(3, random_state=RANDOM_STATE)

,id,total_score_id,title,decrease,base_score,goe,avg_score,unit_id,tournament_id,base_score_total_scores,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,color,school_id,date_start,date_end,origin_id,sequences,cascade,title_nlp,cascade_nlp,multiply,tournament_duration,start_month,end_month,start_day_of_week,end_day_of_week,start_is_weekend,end_is_weekend,start_season,end_season,tournament_year,units_with_experience
75185,85907,14963,CCoSp3V,nC,2.25,0.41,2.66,403,87,24.78,26.68,42.42,19.74,0.0,5,8,Короткая программа,q Прыжок приземлён в четверть x Надбавка за пр...,8,42.42,8,green,198,2090-03-31,2090-04-01,1,"[{'item': 'CCoSp3', 'attr': ['V'], 'fail': None}]",CCoSp3,Change Foot Combination Spin (level 3) reduce...,Change Foot Combination Spin (level 3),0,1,3,4,4,5,0,1,весна,весна,2090,1
170183,321354,460070,CCoSp3V,nC,2.25,0.23,2.48,1858,6866,19.50,26.84,44.89,18.55,-0.5,14,18,Произвольная программа,<< Пониженный прыжок ! Неясное ребро на толчке...,15,73.73,15,green,254,2092-02-19,2092-02-21,0,"[{'item': 'CCoSp3', 'attr': ['V'], 'fail': None}]",CCoSp3,Change Foot Combination Spin (level 3) reduce...,Change Foot Combination Spin (level 3),0,2,2,2,1,3,0,0,зима,зима,2092,1
11120,14017,4889,CCoSp1V,nC,1.50,-0.30,1.20,665,14,30.56,51.84,85.38,33.54,0.0,5,4,Произвольная программа,x Надбавка за прыжки во второй половине програ...,4,126.52,4,lime,65,2090-11-22,2090-11-23,2,"[{'item': 'CCoSp1', 'attr': ['V'], 'fail': None}]",CCoSp1,Change Foot Combination Spin (level 1) reduce...,Change Foot Combination Spin (level 1),0,1,11,11,2,3,0,0,осень,осень,2090,0


In [185]:
data[data["total_score_id"] == 460070]

,id,total_score_id,title,decrease,base_score,goe,avg_score,unit_id,tournament_id,base_score_total_scores,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,color,school_id,date_start,date_end,origin_id,sequences,cascade,title_nlp,cascade_nlp,multiply,tournament_duration,start_month,end_month,start_day_of_week,end_day_of_week,start_is_weekend,end_is_weekend,start_season,end_season,tournament_year,units_with_experience
170179,321350,460070,2F+2Lo,NaN,3.50,-0.66,2.84,1858,6866,19.5,26.84,44.89,18.55,-0.5,14,18,Произвольная программа,<< Пониженный прыжок ! Неясное ребро на толчке...,15,73.73,15,green,254,2092-02-19,2092-02-21,0,"[{'item': '2F', 'attr': [], 'fail': None}, {'i...",2F+2Lo,Double Flip and Double Loop,Double Flip and Double Loop,0,2,2,2,1,3,0,0,зима,зима,2092,1
170180,321351,460070,2Lz!,!,2.10,-0.21,1.89,1858,6866,19.5,26.84,44.89,18.55,-0.5,14,18,Произвольная программа,<< Пониженный прыжок ! Неясное ребро на толчке...,15,73.73,15,green,254,2092-02-19,2092-02-21,0,"[{'item': '2Lz', 'attr': ['!'], 'fail': None}]",2Lz,"Double Lutz not clear edge (Lutz, Flip)",Double Lutz,0,2,2,2,1,3,0,0,зима,зима,2092,1
170181,321352,460070,2F,NaN,1.80,-0.18,1.62,1858,6866,19.5,26.84,44.89,18.55,-0.5,14,18,Произвольная программа,<< Пониженный прыжок ! Неясное ребро на толчке...,15,73.73,15,green,254,2092-02-19,2092-02-21,0,"[{'item': '2F', 'attr': [], 'fail': None}]",2F,Double Flip,Double Flip,0,2,2,2,1,3,0,0,зима,зима,2092,1
170182,321353,460070,2Lz+1Lo,NaN,2.60,-0.21,2.39,1858,6866,19.5,26.84,44.89,18.55,-0.5,14,18,Произвольная программа,<< Пониженный прыжок ! Неясное ребро на толчке...,15,73.73,15,green,254,2092-02-19,2092-02-21,0,"[{'item': '2Lz', 'attr': [], 'fail': None}, {'...",2Lz+1Lo,Double Lutz and Single Loop,Double Lutz and Single Loop,0,2,2,2,1,3,0,0,зима,зима,2092,1
170183,321354,460070,CCoSp3V,nC,2.25,0.23,2.48,1858,6866,19.5,26.84,44.89,18.55,-0.5,14,18,Произвольная программа,<< Пониженный прыжок ! Неясное ребро на толчке...,15,73.73,15,green,254,2092-02-19,2092-02-21,0,"[{'item': 'CCoSp3', 'attr': ['V'], 'fail': None}]",CCoSp3,Change Foot Combination Spin (level 3) reduce...,Change Foot Combination Spin (level 3),0,2,2,2,1,3,0,0,зима,зима,2092,1
170184,321355,460070,2T<<,F,0.00,-0.20,0.24,1858,6866,19.5,26.84,44.89,18.55,-0.5,14,18,Произвольная программа,<< Пониженный прыжок ! Неясное ребро на толчке...,15,73.73,15,green,254,2092-02-19,2092-02-21,0,"[{'item': '2T', 'attr': ['<<'], 'fail': None}]",2T,Double Toeloop downgraded jump / downgraded tw...,Double Toeloop,1,2,2,2,1,3,0,0,зима,зима,2092,1
170185,321356,460070,ChSq1,NaN,3.00,0.00,3.00,1858,6866,19.5,26.84,44.89,18.55,-0.5,14,18,Произвольная программа,<< Пониженный прыжок ! Неясное ребро на толчке...,15,73.73,15,green,254,2092-02-19,2092-02-21,0,"[{'item': 'ChSq1', 'attr': [], 'fail': None}]",ChSq1,Choreo Sequence,Choreo Sequence,0,2,2,2,1,3,0,0,зима,зима,2092,1
170186,321357,460070,1A,NaN,0.00,0.11,1.32,1858,6866,19.5,26.84,44.89,18.55,-0.5,14,18,Произвольная программа,<< Пониженный прыжок ! Неясное ребро на толчке...,15,73.73,15,green,254,2092-02-19,2092-02-21,0,"[{'item': '1A', 'attr': [], 'fail': None}]",1A,Single Axel,Single Axel,1,2,2,2,1,3,0,0,зима,зима,2092,1
170187,321358,460070,FSSp3,NaN,2.60,0.17,2.77,1858,6866,19.5,26.84,44.89,18.55,-0.5,14,18,Произвольная программа,<< Пониженный прыжок ! Неясное ребро на толчке...,15,73.73,15,green,254,2092-02-19,2092-02-21,0,"[{'item': 'FSSp3', 'attr': [], 'fail': None}]",FSSp3,Flying Sit Spin (level 3),Flying Sit Spin (level 3),0,2,2,2,1,3,0,0,зима,зима,2092,1


In [186]:
data[data["decrease"] == "nB"].sample(3, random_state=RANDOM_STATE)

,id,total_score_id,title,decrease,base_score,goe,avg_score,unit_id,tournament_id,base_score_total_scores,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,color,school_id,date_start,date_end,origin_id,sequences,cascade,title_nlp,cascade_nlp,multiply,tournament_duration,start_month,end_month,start_day_of_week,end_day_of_week,start_is_weekend,end_is_weekend,start_season,end_season,tournament_year,units_with_experience
35614,42081,8378,CSSpBV,nB,1.20,0.00,1.20,1075,40,44.53,57.18,106.77,50.59,0.0,8,2,Произвольная программа,REP Повторение прыжка не в каскаде x Надбавка ...,2,158.34,2,lime,264,2091-03-12,2091-03-15,2,"[{'item': 'CSSpB', 'attr': ['V'], 'fail': None}]",CSSpB,Change Foot Sit Spin (level B) reduced value ...,Change Foot Sit Spin (level B),0,3,3,3,0,3,0,0,весна,весна,2091,1
36212,42680,8446,CSSp2V,nB,1.73,0.17,1.90,328,40,52.97,65.21,119.75,55.54,0.0,11,4,Произвольная программа,! Неясное ребро на толчке F/Lz x Надбавка за п...,3,183.91,3,lime,128,2091-03-12,2091-03-15,2,"[{'item': 'CSSp2', 'attr': ['V'], 'fail': None}]",CSSp2,Change Foot Sit Spin (level 2) reduced value ...,Change Foot Sit Spin (level 2),0,3,3,3,0,3,0,0,весна,весна,2091,1
36295,42763,8454,CSSp3V,nB,1.95,0.00,1.95,688,40,37.25,54.65,88.06,33.41,0.0,4,12,Произвольная программа,q Прыжок приземлён в четверть < Недокрученный ...,10,141.34,10,lime,90,2091-03-12,2091-03-15,2,"[{'item': 'CSSp3', 'attr': ['V'], 'fail': None}]",CSSp3,Change Foot Sit Spin (level 3) reduced value ...,Change Foot Sit Spin (level 3),0,3,3,3,0,3,0,0,весна,весна,2091,1


In [187]:
data[data["decrease"] == "nF"].sample(3, random_state=RANDOM_STATE)

,id,total_score_id,title,decrease,base_score,goe,avg_score,unit_id,tournament_id,base_score_total_scores,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,color,school_id,date_start,date_end,origin_id,sequences,cascade,title_nlp,cascade_nlp,multiply,tournament_duration,start_month,end_month,start_day_of_week,end_day_of_week,start_is_weekend,end_is_weekend,start_season,end_season,tournament_year,units_with_experience
129976,217766,449114,CCoSp1V,nF,1.50,-0.50,1.00,4591,6848,6.77,16.80,19.94,4.64,-1.5,20,25,Произвольная программа,REP Повторение прыжка не в каскаде F Падение в...,25,19.94,25,green,192,2091-11-11,2091-11-15,0,"[{'item': 'CCoSp1', 'attr': ['V'], 'fail': None}]",CCoSp1,Change Foot Combination Spin (level 1) reduce...,Change Foot Combination Spin (level 1),0,4,11,11,6,3,1,0,осень,осень,2091,1
69690,79786,14188,CCoSp1V,nF,1.50,-0.03,1.47,1817,83,8.24,27.92,34.31,7.39,0.0,19,25,Произвольная программа,< Недокрученный прыжок << Пониженный прыжок ! ...,25,34.31,25,green,229,2090-04-28,2090-04-29,1,"[{'item': 'CCoSp1', 'attr': ['V'], 'fail': None}]",CCoSp1,Change Foot Combination Spin (level 1) reduce...,Change Foot Combination Spin (level 1),0,1,4,4,4,5,0,1,весна,весна,2090,1
127301,215118,448763,CCoSpBV,nF,1.28,-0.09,1.19,4562,6848,7.06,17.19,22.90,6.21,-0.5,16,13,Произвольная программа,<< Пониженный прыжок REP Повторение прыжка не ...,13,22.90,13,green,256,2091-11-11,2091-11-15,0,"[{'item': 'CCoSpB', 'attr': ['V'], 'fail': None}]",CCoSpB,Change Foot Combination Spin (level B) reduce...,Change Foot Combination Spin (level B),0,4,11,11,6,3,1,0,осень,осень,2091,1


In [188]:
data[data["total_score_id"] == 449114]

,id,total_score_id,title,decrease,base_score,goe,avg_score,unit_id,tournament_id,base_score_total_scores,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,color,school_id,date_start,date_end,origin_id,sequences,cascade,title_nlp,cascade_nlp,multiply,tournament_duration,start_month,end_month,start_day_of_week,end_day_of_week,start_is_weekend,end_is_weekend,start_season,end_season,tournament_year,units_with_experience
129970,217760,449114,1A,F,1.10,-0.55,0.55,4591,6848,6.77,16.8,19.94,4.64,-1.5,20,25,Произвольная программа,REP Повторение прыжка не в каскаде F Падение в...,25,19.94,25,green,192,2091-11-11,2091-11-15,0,"[{'item': '1A', 'attr': [], 'fail': None}]",1A,Single Axel,Single Axel,0,4,11,11,6,3,1,0,осень,осень,2091,1
129971,217761,449114,1Lz,NaN,0.60,0.02,0.62,4591,6848,6.77,16.8,19.94,4.64,-1.5,20,25,Произвольная программа,REP Повторение прыжка не в каскаде F Падение в...,25,19.94,25,green,192,2091-11-11,2091-11-15,0,"[{'item': '1Lz', 'attr': [], 'fail': None}]",1Lz,Single Lutz,Single Lutz,0,4,11,11,6,3,1,0,осень,осень,2091,1
129972,217762,449114,CSSp,NaN,0.00,0.00,0.00,4591,6848,6.77,16.8,19.94,4.64,-1.5,20,25,Произвольная программа,REP Повторение прыжка не в каскаде F Падение в...,25,19.94,25,green,192,2091-11-11,2091-11-15,0,"[{'item': 'CSSp', 'attr': [], 'fail': None}]",CSSp,Change Foot Sit Spin,Change Foot Sit Spin,0,4,11,11,6,3,1,0,осень,осень,2091,1
129973,217763,449114,ChSpl1,NaN,1.50,0.10,1.60,4591,6848,6.77,16.8,19.94,4.64,-1.5,20,25,Произвольная программа,REP Повторение прыжка не в каскаде F Падение в...,25,19.94,25,green,192,2091-11-11,2091-11-15,0,"[{'item': 'ChSpl1', 'attr': [], 'fail': None}]",ChSpl1,Choreo sliding movement,Choreo sliding movement,0,4,11,11,6,3,1,0,осень,осень,2091,1
129974,217764,449114,2S,F,1.30,-0.65,0.65,4591,6848,6.77,16.8,19.94,4.64,-1.5,20,25,Произвольная программа,REP Повторение прыжка не в каскаде F Падение в...,25,19.94,25,green,192,2091-11-11,2091-11-15,0,"[{'item': '2S', 'attr': [], 'fail': None}]",2S,Double Salchow,Double Salchow,0,4,11,11,6,3,1,0,осень,осень,2091,1
129975,217765,449114,1A+REP,F,0.77,-0.55,0.22,4591,6848,6.77,16.8,19.94,4.64,-1.5,20,25,Произвольная программа,REP Повторение прыжка не в каскаде F Падение в...,25,19.94,25,green,192,2091-11-11,2091-11-15,0,"[{'item': '1A', 'attr': [], 'fail': None}, {'i...",1A+REP,Single Axel and Jump Repetition,Single Axel and Jump Repetition,0,4,11,11,6,3,1,0,осень,осень,2091,1
129976,217766,449114,CCoSp1V,nF,1.50,-0.50,1.00,4591,6848,6.77,16.8,19.94,4.64,-1.5,20,25,Произвольная программа,REP Повторение прыжка не в каскаде F Падение в...,25,19.94,25,green,192,2091-11-11,2091-11-15,0,"[{'item': 'CCoSp1', 'attr': ['V'], 'fail': None}]",CCoSp1,Change Foot Combination Spin (level 1) reduce...,Change Foot Combination Spin (level 1),0,4,11,11,6,3,1,0,осень,осень,2091,1


In [189]:
data[data["decrease"].isin(["nS", "nU", "nC", "nB", "nF"])].sample(
    5, random_state=RANDOM_STATE
)

,id,total_score_id,title,decrease,base_score,goe,avg_score,unit_id,tournament_id,base_score_total_scores,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,color,school_id,date_start,date_end,origin_id,sequences,cascade,title_nlp,cascade_nlp,multiply,tournament_duration,start_month,end_month,start_day_of_week,end_day_of_week,start_is_weekend,end_is_weekend,start_season,end_season,tournament_year,units_with_experience
123114,211077,448212,CCoSp2V,nC,1.88,0.00,1.88,110,6844,14.83,19.76,33.29,13.53,0.0,5,7,Короткая программа,* Недопустимый элемент < Недокрученный прыжок ...,7,98.01,7,green,192,2091-10-25,2091-10-26,0,"[{'item': 'CCoSp2', 'attr': ['V'], 'fail': None}]",CCoSp2,Change Foot Combination Spin (level 2) reduce...,Change Foot Combination Spin (level 2),0,1,10,10,3,4,0,0,осень,осень,2091,1
54321,223725,449905,CCoSp1V,nS,1.50,0.15,1.65,1865,6859,21.75,25.68,46.61,20.93,0.0,4,6,Произвольная программа,q Прыжок приземлён в четверть < Недокрученный ...,6,74.04,6,green,25,2092-01-13,2092-01-17,0,"[{'item': 'CCoSp1', 'attr': ['V'], 'fail': None}]",CCoSp1,Change Foot Combination Spin (level 1) reduce...,Change Foot Combination Spin (level 1),0,4,1,1,6,3,1,0,зима,зима,2092,1
22614,27179,6499,CCoSp3V,nS,2.25,0.45,2.70,1024,24,25.04,43.52,69.32,26.80,0.0,23,18,Произвольная программа,< Недокрученный прыжок ! Неясное ребро на толч...,16,137.33,16,green,228,2091-01-26,2091-01-30,2,"[{'item': 'CCoSp3', 'attr': ['V'], 'fail': None}]",CCoSp3,Change Foot Combination Spin (level 3) reduce...,Change Foot Combination Spin (level 3),0,4,1,1,4,1,0,0,зима,зима,2091,0
53013,223650,449895,CCoSp2V,nS,1.88,0.19,2.07,1810,6859,10.41,14.72,25.32,10.60,0.0,15,12,Короткая программа,< Недокрученный прыжок x Надбавка за прыжки во...,12,69.96,12,green,192,2092-01-13,2092-01-17,0,"[{'item': 'CCoSp2', 'attr': ['V'], 'fail': None}]",CCoSp2,Change Foot Combination Spin (level 2) reduce...,Change Foot Combination Spin (level 2),0,4,1,1,6,3,1,0,зима,зима,2092,1
86604,165381,443164,CCoSp2V,nC,1.88,0.23,2.11,837,4789,21.47,22.21,40.99,19.78,-1.0,10,14,Короткая программа,< Недокрученный прыжок ! Неясное ребро на толч...,14,108.85,14,green,193,2091-12-15,2091-12-17,2,"[{'item': 'CCoSp2', 'attr': ['V'], 'fail': None}]",CCoSp2,Change Foot Combination Spin (level 2) reduce...,Change Foot Combination Spin (level 2),0,2,12,12,5,0,1,0,зима,зима,2091,1


Выявлено, что никакого влияния эти отметки не оказывают на базовые очки. Возможно, эти ошибки влияют только на goe от судей. В дальнейшей работе целесообразно глубже изучить систему судейства. В экспериментах предлагается оценить их влияние на goe.

В целом, указанный столбец в некоторой степени дублирует 'title', а также информацию из 'info'. В то же время, замечено, что столбец 'decreasings_score', который ожидаемо может быть связан с 'decrease' и 'info', на самом деле содержит несколько противоречивую информацию.

Предлагается в настоящей работе принять, что все штрафы и снижения очков представлены в столбце 'title' для каждого выполненного элемента, а дополнительный важный параметр - падения, который представлен в столбце 'decrease'.

Предлагается выделить его в отдельный столбец и использовать для отметок о падениях во время выполнения элементов.


In [190]:
data["decrease"] = data["decrease"].fillna("")

In [191]:
if get_processed_data:
    data["falls"] = data["decrease"].progress_apply(lambda x: 1 if "F" in x else 0)

100%|██████████| 159498/159498 [00:00<00:00, 857527.72it/s] 


In [192]:
data["falls"].value_counts()

falls
0    150227
1      9271
Name: count, dtype: int64

## Изучение поля "components_score"

Сформируем новый столбец, отражающий очки за артистизм, как среднее между всеми элементами в одном выступлении

In [193]:
counts = data.groupby("total_score_id")["total_score_id"].transform("count")

In [194]:
if get_processed_data:
    data["components_score_per_element"] = data["components_score"] / counts

## Изучение поля "base_score"

Сформируем новое поле, рассчитанное, как разность 'avg_score' и 'goe'

In [195]:
if get_processed_data:
    data["custom_base_score"] = data["avg_score"] - data["goe"]

## Создание агрегированных данных по соревнованиям

Агрегацию данных мы будем производить с учетом особенностей временного ряда - занимаемые места, средние очки от соревнования к соревнованию могут улучшаться. 

В результате экспериментов выбрано окно времени, равное одному году. Как правило, прогресс спортсмена заметен на этом периоде.

Так как агрегация будет производиться по спортсменам, то наличие спортсменов в данных, которые позднее будут удалены - не критично. Для таких спортсменов агрегированные данные будет некорректными, но подлежать удалению. Они оставлены для сравнения результатов обучения моделей.

In [196]:
# Создание признака среднего среди всех предыдущих соревнований за прошедший календарный год
def calculate_avg_last_year(group, column):
    averages = []
    for i in range(len(group)):
        current_date = group.iloc[i]["date_start"]
        start_date = current_date - timedelta(days=365)
        previous_year_tournaments = group[
            (group["date_start"] >= start_date) & (group["date_start"] < current_date)
        ]
        if not previous_year_tournaments.empty:
            avg = previous_year_tournaments[column].mean()
        else:
            avg = np.nan
        averages.append(avg)
    return pd.Series(averages, index=group.index)

In [197]:
if get_processed_data:
    # Агрегирование данных на уровне соревнования
    agg_data = (
        data.groupby(["unit_id", "tournament_id"])
        .agg(
            date_start=("date_start", "first"),
            overall_place=("overall_place", "first"),
            overall_total_score=("overall_total_score", "first"),
        )
        .reset_index()
    )

    # Сортировка по дате для каждого спортсмена
    agg_data = agg_data.sort_values(by=["unit_id", "date_start"])

    agg_data["avg_overall_place_last_year"] = (
        data.groupby("unit_id")
        .progress_apply(calculate_avg_last_year, column="overall_place")
        .reset_index(level=0, drop=True)
    )

    agg_data["avg_overall_total_score_last_year"] = (
        agg_data.groupby("unit_id")
        .progress_apply(calculate_avg_last_year, column="overall_total_score")
        .reset_index(level=0, drop=True)
    )

    data = pd.merge(
        data,
        agg_data[
            [
                "unit_id",
                "tournament_id",
                "avg_overall_place_last_year",
                "avg_overall_total_score_last_year",
            ]
        ],
        on=["unit_id", "tournament_id"],
    )

100%|██████████| 3032/3032 [00:08<00:00, 357.04it/s]


Часть данных на соревновании зависят от программы: "Короткая" или "Произвольная". Агрегация для этих данных несколько иная.

In [198]:
if get_processed_data:
    # Агрегирование данных на уровне соревнования
    agg_data = (
        data.groupby(["unit_id", "tournament_id", "segment_name"])
        .agg(
            date_start=("date_start", "first"),
            components_score=("components_score", "first"),
            place=("place", "first"),
            elements_score=("elements_score", "first"),
            decreasings_score=("decreasings_score", "first"),
            total_score=("total_score", "first"),
            falls=("falls", "sum"),  # Предположим, что падения суммируются по элементам
        )
        .reset_index()
    )

    # Сортировка по дате для каждого спортсмена
    agg_data = agg_data.sort_values(by=["unit_id", "date_start", "segment_name"])

    agg_data["avg_components_score_last_year"] = (
        agg_data.groupby(["unit_id", "segment_name"])
        .progress_apply(calculate_avg_last_year, column="components_score")
        .reset_index(level=[0, 1], drop=True)
    )

    agg_data["avg_place_last_year"] = (
        agg_data.groupby(["unit_id", "segment_name"])
        .progress_apply(calculate_avg_last_year, column="place")
        .reset_index(level=[0, 1], drop=True)
    )

    agg_data["avg_elements_score_last_year"] = (
        agg_data.groupby(["unit_id", "segment_name"])
        .progress_apply(calculate_avg_last_year, column="elements_score")
        .reset_index(level=[0, 1], drop=True)
    )

    agg_data["avg_decreasings_score_last_year"] = (
        agg_data.groupby(["unit_id", "segment_name"])
        .progress_apply(calculate_avg_last_year, column="decreasings_score")
        .reset_index(level=[0, 1], drop=True)
    )

    agg_data["avg_total_score_last_year"] = (
        agg_data.groupby(["unit_id", "segment_name"])
        .progress_apply(calculate_avg_last_year, column="total_score")
        .reset_index(level=[0, 1], drop=True)
    )

    agg_data["avg_falls_last_year"] = (
        agg_data.groupby(["unit_id", "segment_name"])
        .progress_apply(calculate_avg_last_year, column="falls")
        .reset_index(level=[0, 1], drop=True)
    )

    data = pd.merge(
        data,
        agg_data[
            [
                "unit_id",
                "tournament_id",
                "segment_name",
                "avg_components_score_last_year",
                "avg_place_last_year",
                "avg_elements_score_last_year",
                "avg_decreasings_score_last_year",
                "avg_total_score_last_year",
                "avg_falls_last_year",
            ]
        ],
        on=["unit_id", "tournament_id", "segment_name"],
    )

100%|██████████| 4908/4908 [00:14<00:00, 329.26it/s]


In [199]:
data.shape

(159498, 53)

## Изучение поля "sequences"

Сколько максимальное количество элементов в наборе.

In [200]:
max_n = 0
for sequence in data["sequences"]:
    if len(sequence) > max_n:
        max_n = len(sequence)
max_n

5

## Формирование ошибок

In [201]:
def get_target_clear_element(sequence):
    for element in sequence:
        if element["attr"] != []:
            return 0
    return 1

In [202]:
if get_processed_data:
    data["target_clear_element"] = data["sequences"].progress_apply(
        get_target_clear_element
    )

100%|██████████| 159498/159498 [00:00<00:00, 672984.67it/s]


In [203]:
def get_max_base_score(sequences, base_scores_dict):
    max_base_score = 0
    for item in sequences:
        try:
            max_base_score += base_scores_dict[
                base_scores_dict["item"] == item["item"]
            ].iloc[0, 1]
        except IndexError:
            max_base_score += 0
    return max_base_score

In [204]:
if get_processed_data:
    max_base_score = (
        data["sequences"]
        .progress_apply(get_max_base_score, args=(base_scores_dict,))
        .max()
    )

100%|██████████| 159498/159498 [00:47<00:00, 3361.24it/s]


In [205]:
if get_processed_data:
    max_base_score

In [206]:
def get_difficulty(row, base_scores_dict):
    sequences = row["sequences"]
    multiply = row["multiply"]
    base_score = 0
    for item in sequences:
        try:
            base_score += base_scores_dict[
                base_scores_dict["item"] == item["item"]
            ].iloc[0, 1]
        except IndexError:
            base_score += 0
    max_base_score = 18.1

    return round(base_score * (1 + 0.1 * multiply) / max_base_score, 2)

In [207]:
if get_processed_data:
    data["difficulty"] = data.progress_apply(
        get_difficulty, args=(base_scores_dict,), axis=1
    )

100%|██████████| 159498/159498 [00:52<00:00, 3030.91it/s]


In [208]:
data.shape

(159498, 55)

In [209]:
def get_errors(sequences):
    errors = []
    for sequence in sequences:
        errors += sequence["attr"]
    errors = set(errors)
    mistakes = {
        0: 0b1000000,
        "q": 0b0100000,
        "e": 0b0010000,
        "<": 0b0001000,
        "<<": 0b0000100,
        "!": 0b0000010,
        "V": 0b0000001,
    }
    errors_element = 0
    if len(errors) == 0:
        errors_element += mistakes[0]
    for error in errors:
        errors_element += mistakes[error]
    error = format(errors_element, "b").zfill(7)

    return pd.Series(list(error))

In [210]:
if get_processed_data:
    # if True:
    result = data["sequences"].progress_apply(get_errors)
    result = result.rename(
        columns={
            0: f"perfect_element",
            1: f"q_element",
            2: f"e_element",
            3: f"l_element",
            4: f"ll_element",
            5: f"h_element",
            6: f"v_element",
        }
    )

    data = pd.concat([data, result], axis=1)

    data.shape

100%|██████████| 159498/159498 [00:12<00:00, 12509.91it/s]


In [211]:
if get_processed_data:
    # if True:
    int_cols = data.columns[55:62]

    for col in int_cols:
        data[col] = data[col].astype(int)

In [217]:
if get_processed_data:
    # if True:

    joblib.dump(data, "../data/processed/processed_data.joblib")
    get_processed_data = False

In [289]:
small_data = data[data["unit_id"] == 2317]
small_data.shape
joblib.dump(small_data, "../data/processed/data.joblib")


['../data/processed/data.joblib']

In [218]:
data[data["units_with_experience"] == 0].shape

(23017, 62)

In [219]:
data.head(2)

,id,total_score_id,title,decrease,base_score,goe,avg_score,unit_id,tournament_id,base_score_total_scores,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,color,school_id,date_start,date_end,origin_id,sequences,cascade,title_nlp,cascade_nlp,multiply,tournament_duration,start_month,end_month,start_day_of_week,end_day_of_week,start_is_weekend,end_is_weekend,start_season,end_season,tournament_year,units_with_experience,falls,components_score_per_element,custom_base_score,avg_overall_place_last_year,avg_overall_total_score_last_year,avg_components_score_last_year,avg_place_last_year,avg_elements_score_last_year,avg_decreasings_score_last_year,avg_total_score_last_year,avg_falls_last_year,target_clear_element,difficulty,perfect_element,q_element,e_element,l_element,ll_element,h_element,v_element
0,1,1,2A,,3.3,0.66,3.96,1,1,31.99,24.67,56.86,33.19,0.0,2,5,Короткая программа,x Надбавка за прыжки во второй половине програ...,4,164.44,4,green,198,2090-11-29,2090-12-01,2,"[{'item': '2A', 'attr': [], 'fail': None}]",2A,Double Axel,Double Axel,0,2,11,12,2,4,0,0,осень,зима,2090,1,0,3.524286,3.3,13.384615,103.855,24.47,20.0,25.465,0.0,48.435,0.0,1,0.18,1,0,0,0,0,0,0
1,2,1,3F+3Lo,,10.2,-0.11,10.09,1,1,31.99,24.67,56.86,33.19,0.0,2,5,Короткая программа,x Надбавка за прыжки во второй половине програ...,4,164.44,4,green,198,2090-11-29,2090-12-01,2,"[{'item': '3F', 'attr': [], 'fail': None}, {'i...",3F+3Lo,Triple Flip and Triple Loop,Triple Flip and Triple Loop,0,2,11,12,2,4,0,0,осень,зима,2090,1,0,3.524286,10.2,13.384615,103.855,24.47,20.0,25.465,0.0,48.435,0.0,1,0.56,1,0,0,0,0,0,0


In [220]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159498 entries, 0 to 159497
Data columns (total 62 columns):
 #   Column                             Non-Null Count   Dtype         
---  ------                             --------------   -----         
 0   id                                 159498 non-null  int64         
 1   total_score_id                     159498 non-null  int64         
 2   title                              159498 non-null  object        
 3   decrease                           159498 non-null  object        
 4   base_score                         159498 non-null  float64       
 5   goe                                159498 non-null  float64       
 6   avg_score                          159498 non-null  float64       
 7   unit_id                            159498 non-null  int64         
 8   tournament_id                      159498 non-null  int64         
 9   base_score_total_scores            159498 non-null  float64       
 10  components_score    

## Формирование эмбеддингов

Сколько максимальное количество элементов в наборе.

In [221]:
if get_nlp_dict:
    add_nlp_title = data[["title", "title_nlp"]].rename(
        columns={"title": "item", "title_nlp": "nlp_item"}
    )
    display(add_nlp_title.head())

,item,nlp_item
0,2A,Double Axel
1,3F+3Lo,Triple Flip and Triple Loop
2,3Lz,Triple Lutz
3,CCoSp4,Change Foot Combination Spin (level 4)
4,FCSp4,Flying Camel Spin (level 4)


In [222]:
if get_nlp_dict:
    add_nlp_cascade = data[["cascade", "cascade_nlp"]].rename(
        columns={"cascade": "item", "cascade_nlp": "nlp_item"}
    )
    display(add_nlp_cascade.head())

,item,nlp_item
0,2A,Double Axel
1,3F+3Lo,Triple Flip and Triple Loop
2,3Lz,Triple Lutz
3,CCoSp4,Change Foot Combination Spin (level 4)
4,FCSp4,Flying Camel Spin (level 4)


In [223]:
nlp_dict.shape

(168, 2)

In [224]:
if get_nlp_dict:
    nlp_dict = pd.concat([nlp_dict, add_nlp_title, add_nlp_cascade]).drop_duplicates()

In [225]:
nlp_dict.shape

(3299, 2)

In [226]:
if get_nlp_dict:
    # Проверка доступности GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

Using device: cpu


In [227]:
if get_nlp_dict:
    # Загрузка предобученной модели и токенизатора
    tokenizer = BertTokenizer.from_pretrained("distilbert-base-uncased")
    model = BertModel.from_pretrained("distilbert-base-uncased").to(device)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertModel were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 

In [228]:
# Получение эмбеддинга специального токена [CLS]
def get_cls_embedding(sequence):
    inputs = tokenizer(sequence, return_tensors="pt", padding=True, truncation=True).to(
        device
    )
    with torch.no_grad():
        outputs = model(**inputs)
    # Эмбеддинги из последнего скрытого состояния
    embeddings = outputs.last_hidden_state
    # Эмбеддинг специального токена [CLS] находится на позиции 0
    cls_embedding = embeddings[:, 0, :].squeeze().cpu().numpy()
    return cls_embedding

In [229]:
if get_nlp_dict:
    # Применение к списку последовательностей
    embeddings = []
    for seq in tqdm(nlp_dict["nlp_item"]):
        embeddings.append(get_cls_embedding(seq))

100%|██████████| 3299/3299 [02:49<00:00, 19.48it/s]


In [230]:
if get_nlp_dict:
    # Применение PCA для снижения размерности
    pca_10 = PCA(n_components=10)
    reduced_embeddings_10 = pca_10.fit_transform(embeddings)

In [231]:
if get_nlp_dict:
    emb_columns = ["embed_" + str(x) for x in range(0, 10)]

In [232]:
if get_nlp_dict:
    df_embeddings = pd.DataFrame(
        reduced_embeddings_10, columns=emb_columns, index=nlp_dict.index
    )
    df_embeddings.head()

In [233]:
if get_nlp_dict:
    df_embeddings.shape

In [234]:
nlp_dict.shape

(3299, 2)

In [235]:
if get_nlp_dict:
    nlp_dict = pd.concat([nlp_dict, df_embeddings], axis=1)

In [236]:
nlp_dict.shape

(3299, 12)

In [237]:
if get_nlp_dict:
    joblib.dump(nlp_dict, "../data/mats/nlp_dict.joblib")

# Преобразование данных в поэлементный разбор

In [238]:
def process_row(row):
    sequence = row["sequences"]
    result_rows = []
    for i, element in enumerate(sequence):
        new_row = row.copy()
        new_row["element"] = element["item"]

        new_row["attr_element"] = "".join(element["attr"]) if element["attr"] else 0

        if i - 1 >= 0:
            prev_element = sequence[i - 1]
            new_row["prev_element"] = prev_element["item"]
            new_row["attr_prev_element"] = (
                "".join(prev_element["attr"]) if prev_element["attr"] else 0
            )
        else:
            new_row["prev_element"] = 0
            new_row["attr_prev_element"] = 0

        if i + 1 < len(sequence):
            next_element = sequence[i + 1]
            new_row["next_element"] = next_element["item"]
            new_row["attr_next_element"] = (
                "".join(next_element["attr"]) if next_element["attr"] else 0
            )
        else:
            new_row["next_element"] = 0
            new_row["attr_next_element"] = 0

        new_row["single_element"] = (
            1 if new_row["prev_element"] == "wo" and new_row["next_element"] == 0 else 0
        )

        result_rows.append(new_row)

    return result_rows

In [239]:
file_path = "../data/processed/elements_data.joblib"

if os.path.isfile(file_path):
    elements_data = joblib.load(file_path)
    create_elements_data = False
else:
    create_elements_data = True

In [240]:
if create_elements_data:
    result = []
    for index, row in tqdm(data.iterrows(), total=data.shape[0]):
        result.extend(process_row(row))

    elements_data = pd.DataFrame(result).reset_index(drop=True)

100%|██████████| 159498/159498 [07:50<00:00, 339.20it/s]


In [241]:
def get_target_clear_element(attrs):
    if attrs == 0:
        return 1
    else:
        return 0

In [242]:
if create_elements_data:
    elements_data["target_clear_element"] = elements_data[
        "attr_element"
    ].progress_apply(get_target_clear_element)

100%|██████████| 206464/206464 [00:00<00:00, 889937.00it/s] 


In [243]:
def get_clear_prev_element(row):
    prev_element = row["prev_element"]
    attr_prev_element = row["attr_prev_element"]
    if prev_element == 0:
        return 0
    elif attr_prev_element == 0:
        return 0
    else:
        return 1

In [244]:
if create_elements_data:
    elements_data["clear_prev_element"] = elements_data.progress_apply(
        get_clear_prev_element, axis=1
    )

100%|██████████| 206464/206464 [00:02<00:00, 87250.42it/s] 


In [245]:
def get_clear_next_element(row):
    next_element = row["next_element"]
    attr_next_element = row["attr_next_element"]
    if next_element == 0:
        return 0
    elif attr_next_element == 0:
        return 0
    else:
        return 1

In [246]:
if create_elements_data:
    elements_data["clear_next_element"] = elements_data.progress_apply(
        get_clear_next_element, axis=1
    )

100%|██████████| 206464/206464 [00:02<00:00, 90640.89it/s] 


In [247]:
base_scores_dict["base_score"].max()

12.5

In [248]:
def get_difficulty(row, base_scores_dict):
    element = row["element"]
    multiply = row["multiply"]
    try:
        base_score = base_scores_dict[base_scores_dict["item"] == element].iloc[0, 1]
    except IndexError:
        base_score = -1000
    max_scores = 12.5

    return round(base_score * (1 + 0.1 * multiply) / max_scores, 2)

In [249]:
if create_elements_data:
    elements_data["difficulty"] = elements_data.progress_apply(
        get_difficulty, args=(base_scores_dict,), axis=1
    )

100%|██████████| 206464/206464 [00:53<00:00, 3855.39it/s]


In [250]:
if create_elements_data:
    elements_data = elements_data[~(elements_data["difficulty"] < 0)]

In [251]:
elements_data["attr_element"].unique()

array([0, 'q', '<', '!<<', '<<', 'V', '!', 'e', '!q', '!<', 'eq', 'e<<',
       'e<'], dtype=object)

In [252]:
elements_data["prev_element"] = elements_data["prev_element"].astype(str)
elements_data["next_element"] = elements_data["next_element"].astype(str)
elements_data["element"] = elements_data["element"].astype(str)
elements_data["attr_element"] = elements_data["attr_element"].astype(str)
elements_data["attr_prev_element"] = elements_data["attr_prev_element"].astype(str)
elements_data["attr_next_element"] = elements_data["attr_next_element"].astype(str)

In [253]:
def get_mistakes(attr_element):
    mistakes = {
        "0": "1000000",
        "q": "0100000",
        "<": "0001000",
        "!<<": "0000110",
        "<<": "0000100",
        "V": "0000001",
        "!": "0000010",
        "e": "0010000",
        "!q": "0100010",
        "!<": "0001010",
        "eq": "0110000",
        "e<<": "0010100",
        "e<": "0011000",
    }

    return pd.Series(list(mistakes[attr_element]))

In [254]:
if create_elements_data:
    # if True:
    for type_element in ["attr_element", "attr_prev_element", "attr_next_element"]:
        result = elements_data[type_element].progress_apply(get_mistakes)
        result = result.rename(
            columns={
                0: f"perfect_{type_element}",
                1: f"q_{type_element}",
                2: f"e_{type_element}",
                3: f"l_{type_element}",
                4: f"ll_{type_element}",
                5: f"h_{type_element}",
                6: f"v_{type_element}",
            }
        )

        elements_data = pd.concat([elements_data, result], axis=1)

    elements_data.shape

100%|██████████| 197108/197108 [00:15<00:00, 13111.61it/s]


In [255]:
elements_data["attr_element"].unique()

array(['0', 'q', '<', '!<<', '<<', 'V', '!', 'e', '!q', '!<', 'eq', 'e<<',
       'e<'], dtype=object)

In [256]:
elements_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 197108 entries, 0 to 206463
Data columns (total 92 columns):
 #   Column                             Non-Null Count   Dtype         
---  ------                             --------------   -----         
 0   id                                 197108 non-null  int64         
 1   total_score_id                     197108 non-null  int64         
 2   title                              197108 non-null  object        
 3   decrease                           197108 non-null  object        
 4   base_score                         197108 non-null  float64       
 5   goe                                197108 non-null  float64       
 6   avg_score                          197108 non-null  float64       
 7   unit_id                            197108 non-null  int64         
 8   tournament_id                      197108 non-null  int64         
 9   base_score_total_scores            197108 non-null  float64       
 10  components_score         

In [257]:
if create_elements_data:
    # if True:
    int_cols = elements_data.columns[71:92]

    for col in int_cols:
        elements_data[col] = elements_data[col].astype(int)

In [258]:
if create_elements_data:
    # if True:
    joblib.dump(elements_data, "../data/processed/elements_data.joblib")

    create_elements_data = False

In [259]:
elements_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 197108 entries, 0 to 206463
Data columns (total 92 columns):
 #   Column                             Non-Null Count   Dtype         
---  ------                             --------------   -----         
 0   id                                 197108 non-null  int64         
 1   total_score_id                     197108 non-null  int64         
 2   title                              197108 non-null  object        
 3   decrease                           197108 non-null  object        
 4   base_score                         197108 non-null  float64       
 5   goe                                197108 non-null  float64       
 6   avg_score                          197108 non-null  float64       
 7   unit_id                            197108 non-null  int64         
 8   tournament_id                      197108 non-null  int64         
 9   base_score_total_scores            197108 non-null  float64       
 10  components_score         

In [260]:
elements_data.isna().sum()

id                                       0
total_score_id                           0
title                                    0
decrease                                 0
base_score                               0
goe                                      0
avg_score                                0
unit_id                                  0
tournament_id                            0
base_score_total_scores                  0
components_score                         0
total_score                              0
elements_score                           0
decreasings_score                        0
starting_place                           0
place                                    0
segment_name                             0
info                                     0
overall_place                            0
overall_total_score                      0
overall_place_str                        0
color                                    0
school_id                                0
date_start 

In [261]:
elements_data[elements_data["title"].str.contains("V")].head()

,id,total_score_id,title,decrease,base_score,goe,avg_score,unit_id,tournament_id,base_score_total_scores,components_score,total_score,elements_score,decreasings_score,starting_place,place,segment_name,info,overall_place,overall_total_score,overall_place_str,color,school_id,date_start,date_end,origin_id,sequences,cascade,title_nlp,cascade_nlp,multiply,tournament_duration,start_month,end_month,start_day_of_week,end_day_of_week,start_is_weekend,end_is_weekend,start_season,end_season,tournament_year,units_with_experience,falls,components_score_per_element,custom_base_score,avg_overall_place_last_year,avg_overall_total_score_last_year,avg_components_score_last_year,avg_place_last_year,avg_elements_score_last_year,avg_decreasings_score_last_year,avg_total_score_last_year,avg_falls_last_year,target_clear_element,difficulty,perfect_element,q_element,e_element,l_element,ll_element,h_element,v_element,element,attr_element,prev_element,attr_prev_element,next_element,attr_next_element,single_element,clear_prev_element,clear_next_element,perfect_attr_element,q_attr_element,e_attr_element,l_attr_element,ll_attr_element,h_attr_element,v_attr_element,perfect_attr_prev_element,q_attr_prev_element,e_attr_prev_element,l_attr_prev_element,ll_attr_prev_element,h_attr_prev_element,v_attr_prev_element,perfect_attr_next_element,q_attr_next_element,e_attr_next_element,l_attr_next_element,ll_attr_next_element,h_attr_next_element,v_attr_next_element
75,60,57,CCoSp3V,nS,2.25,0.08,2.33,4,1,24.18,27.69,53.84,26.15,0.0,4,2,Короткая программа,x Надбавка за прыжки во второй половине програ...,2,145.05,2,lime,246,2090-11-29,2090-12-01,2,"[{'item': 'CCoSp3', 'attr': ['V'], 'fail': None}]",CCoSp3,Change Foot Combination Spin (level 3) reduce...,Change Foot Combination Spin (level 3),0,2,11,12,2,4,0,0,осень,зима,2090,1,0,3.955714,2.25,4.381818,126.23,27.38,17.0,15.58,0.0,41.96,1.0,0,0.24,0,0,0,0,0,0,1,CCoSp3,V,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0
216,167,164,CCoSp3V,nS,2.25,0.53,2.78,10,1,16.25,21.75,36.85,16.10,0.0,22,21,Короткая программа,< Недокрученный прыжок << Пониженный прыжок x ...,22,99.10,22,green,23,2090-11-29,2090-12-01,2,"[{'item': 'CCoSp3', 'attr': ['V'], 'fail': None}]",CCoSp3,Change Foot Combination Spin (level 3) reduce...,Change Foot Combination Spin (level 3),0,2,11,12,2,4,0,0,осень,зима,2090,0,0,3.107143,2.25,16.000000,41.16,25.42,27.0,15.74,0.0,41.16,0.0,0,0.24,0,0,0,0,0,0,1,CCoSp3,V,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0
301,232,223,FCCoSp2V,nS,1.88,0.06,1.94,13,1,29.75,43.86,72.77,28.91,0.0,4,2,Произвольная программа,q Прыжок приземлён в четверть < Недокрученный ...,2,107.55,2,lime,97,2090-11-29,2090-12-01,2,"[{'item': 'FCCoSp2', 'attr': ['V'], 'fail': No...",FCCoSp2,Flying Change Foot Comb. Spin (level 2) reduc...,Flying Change Foot Comb. Spin (level 2),0,2,11,12,2,4,0,0,осень,зима,2090,0,0,4.386000,1.88,16.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.20,0,0,0,0,0,0,1,FCCoSp2,V,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0
451,346,342,CCoSp3V,nS,2.25,0.15,2.40,20,1,32.44,36.51,71.23,34.72,0.0,17,10,Произвольная программа,<< Пониженный прыжок ! Неясное ребро на толчке...,8,117.15,8,green,249,2090-11-29,2090-12-01,2,"[{'item': 'CCoSp3', 'attr': ['V'], 'fail': None}]",CCoSp3,Change Foot Combination Spin (level 3) reduce...,Change Foot Combination Spin (level 3),0,2,11,12,2,4,0,0,осень,зима,2090,1,0,4.563750,2.25,10.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.24,0,0,0,0,0,0,1,CCoSp3,V,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0
535,413,407,CCoSp3V,nS,2.25,0.08,2.33,24,1,35.29,37.62,75.39,37.77,0.0,11,7,Произвольная программа,x Надбавка за прыжки во второй половине програ...,10,114.46,10,green,97,2090-11-29,2090-12-01,2,"[{'item': 'CCoSp3', 'attr': ['V'], 'fail': None}]",CCoSp3,Change Foot Combination Spin (level 3) reduce...,Change Foot Combination Spin (level 3),0,2,11,12,2,4,0,0,осень,зима,2090,1,0,3.762000,2.25,12.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.24,0,0,0,0,0,0,1,CCoSp3,V,0,0,0,0,0,0,0,0,

# VALIDATION DATA

**Валидация моделей:**
   - Общая валидация качества моделей для их отбора будет производиться на основании разделения выборки на тренировочную, валидационную и тестовую выборки в соотношении (70:20:10).
   - Тестовую выборку применим только к итоговым моделям.
   - Для итоговой "эмпирической" валидации моделей используем данные 3-х рандомных спортсменов.
   - Для итогового "эмпирического" тестирования выберем также данные других 3-х рандомных спортсменов.

## data

In [262]:
final_data = data[~(data["units_with_experience"] == 0)]
final_data = final_data.fillna(0)

In [263]:
final_data.shape

(136481, 62)

ID спортсмена для тестирования "нового" спортсмена. Данные, связанные с указанным спортсменом необходимо полностью "изъять".

In [264]:
random.seed(RANDOM_STATE)
final_test_unit = random.sample(sorted(final_data["unit_id"].unique()), 1)
final_test_unit

[1112]

In [265]:
final_test_unit_data = final_data[(final_data["unit_id"].isin(final_test_unit))]
final_test_unit_data.shape

(41, 62)

In [266]:
final_data_1 = final_data[~(final_data["unit_id"].isin(final_test_unit))]
final_data_1.shape

(136440, 62)

ID пользователей для валидации. Выберем трех пользователей и "отложим" данные о их последнем соревновании. При этом выберем данные за последнее соревнование в данных.

In [267]:
last_tournament = final_data["date_start"].sort_values(ascending=False).unique()[0]
last_tournament

Timestamp('2092-04-29 00:00:00')

In [268]:
ids_for_validation = final_data_1[final_data_1["date_start"] == last_tournament][
    "unit_id"
].unique()
ids_for_validation

array([  227,   229,   220,  2032,  9820,  2023,  2230,  2127,  2235,
        4017,  4008,  3893,  4114,  4104,  9809,  4013,  9767,  3853,
        3867,  3863,  2268,  1895,   202,  4132,  1825,  2152,   174,
        2401,  2117,  3908,  9791,   228,  2125,   181,  2124,  2404,
        9450,    93,  1805,  2118,  2017,   190,  3859,  2129,  3909,
         193,  2237,  1848,  1965,  1928,  1969,  1808,  4091,  2024,
        1817,  2198,  2288,  1856,  2073,  2103,  3872,  3852,  2018,
        2104,  4551,  1854,  1995,  9992,  3878, 34922,  4992,  2948,
        2093,  5096,  1968,   173,  9973,  2082,  3210,  2951,  3213,
        2415,  4549,  3212,  1886,   200,  9787,  1887,  2437,  4087,
       10001,   204,  3291,  3297,  4312,  4331,  4314,    18,  2173,
         257,   132,   270,  2383,  2861,  9466,  4615,  2061,   883,
         110,  2637,   215,   211,  2109,   168,  9936,  2887,  2889,
        2884,  9797,  4424,  9993,  1763,  2955,  3902,  3877,  3847,
        2133,  2138,

In [269]:
random.seed(RANDOM_STATE)
validation_unit = random.sample(sorted(ids_for_validation), 6)
validation_unit

[2317, 3893, 2147, 2131, 202, 3908]

In [270]:
valid_unit_data = final_data_1[(final_data_1["unit_id"].isin(validation_unit[0:3]))]
valid_unit_data.shape

(115, 62)

In [271]:
test_unit_data = final_data_1[(final_data_1["unit_id"].isin(validation_unit[3:6]))]
test_unit_data.shape

(199, 62)

In [272]:
final_data_2 = final_data_1[~(final_data_1["unit_id"].isin(validation_unit))]
final_data_2.shape

(136126, 62)

## elements_data

In [273]:
final_elements_data = elements_data[~(elements_data["units_with_experience"] == 0)]
final_elements_data = final_elements_data.fillna(0)

In [274]:
final_elements_data.shape

(169177, 92)

In [275]:
final_test_unit_elements_data = final_elements_data[
    (final_elements_data["unit_id"].isin(final_test_unit))
]
final_test_unit_elements_data.shape

(51, 92)

In [276]:
final_elements_data_1 = final_elements_data[
    ~(final_elements_data["unit_id"].isin(final_test_unit))
]
final_elements_data_1.shape

(169126, 92)

In [277]:
valid_unit_elements_data = final_elements_data_1[
    (final_elements_data_1["unit_id"].isin(validation_unit[0:3]))
]
valid_unit_elements_data.shape

(138, 92)

In [278]:
test_unit_elements_data = final_elements_data_1[
    (final_elements_data_1["unit_id"].isin(validation_unit[3:6]))
]
test_unit_elements_data.shape

(243, 92)

In [279]:
final_elements_data_2 = final_elements_data_1[
    ~(final_elements_data_1["unit_id"].isin(validation_unit))
]
final_elements_data_2.shape

(168745, 92)

In [280]:
joblib.dump(final_data_2, "../data/processed/final_data.joblib")

['../data/processed/final_data.joblib']

In [281]:
joblib.dump(final_elements_data_2, "../data/processed/final_elements_data.joblib")

['../data/processed/final_elements_data.joblib']

In [282]:
joblib.dump(final_elements_data_2, "../data/processed/final_elements_data.joblib")

['../data/processed/final_elements_data.joblib']

In [283]:
joblib.dump(final_test_unit_data, "../data/processed/final_test_unit_data")
joblib.dump(valid_unit_data, "../data/processed/valid_unit_data")
joblib.dump(test_unit_data, "../data/processed/test_unit_data")

['../data/processed/test_unit_data']

In [284]:
joblib.dump(
    final_test_unit_elements_data, "../data/processed/final_test_unit_elements_data"
)
joblib.dump(valid_unit_elements_data, "../data/processed/valid_unit_elements_data")
joblib.dump(test_unit_elements_data, "../data/processed/test_unit_elements_data")


['../data/processed/test_unit_elements_data']

# Вывод по преобработке данных

Итоговые данные, готовые для моделирования, включают:

- final_elements_data: Данные о выполненных элементах фигурного катания, включая закодированные и масштабированные признаки.
- final_test_unit_data: Тестовые данные для прогнозирования выполнения элементов.
- valid_unit_data: Данные для валидации моделей.
- test_unit_elements_data: Данные для финального тестирования моделей.

Проведенная работа по предобработке данных позволила подготовить качественные и структурированные данные для обучения прогностической модели. Были обработаны пропущенные значения, созданы новые признаки, исправлены выбросы и ошибки, а также выполнено разделение данных на обучающие и тестовые наборы. Эти шаги являются основой для разработки надежной и точной модели, которая поможет предсказывать выполнение элементов фигурного катания и, соответственно, улучшать результаты тренировок спортсменов.